# qecsim demos

## Simulating error correction with a rotated planar stabilizer code
This demo shows verbosely how to simulate one error correction run. 

| For normal use, the code in this demo is encapsulated in the function:
| `qecsim.app.run_once(code, error_model, decoder, error_probability)`,
| and the simulation of many error correction runs is encapsulated in the function:
| `qecsim.app.run(code, error_model, decoder, error_probability, max_runs, max_failures)`.

Notes:

* Operators can be visualised in binary symplectic form (bsf) or Pauli form, e.g. `[1 1 0|0 1 0] = XYI`.
* The binary symplectic product is denoted by $\odot$ and defined as $A \odot B \equiv A \Lambda B \bmod 2$ where $\Lambda = \left[\begin{matrix} 0 & I \\ I & 0 \end{matrix}\right]$.
* Binary addition is denoted by $\oplus$ and defined as addition modulo 2, or equivalently exclusive-or.

### Initialise the models

In [4]:
%run qsu.ipynb  # color-printing functions
import collections
import itertools
import numpy as np
from qecsim import paulitools as pt
import matplotlib.pyplot as plt
import qecsim
from qecsim import app
from qecsim.models.generic import PhaseFlipErrorModel,DepolarizingErrorModel,BiasedDepolarizingErrorModel
from qecsim.models.planar import PlanarCode, PlanarMPSDecoder
from qecsim.models.rotatedplanar import RotatedPlanarCode, RotatedPlanarMPSDecoder

from _planarmpsdecoder_def import PlanarMPSDecoder_def
from _rotatedplanarmpsdecoder_def import RotatedPlanarMPSDecoder_def
import app_def
import importlib as imp
imp.reload(app_def)
import os, time
import multiprocessing as mp
from functools import partial
from itertools import cycle

# initialise models
my_code = RotatedPlanarCode(7, 7)
my_error_model = DepolarizingErrorModel()
my_decoder = RotatedPlanarMPSDecoder(chi=8)
# print models
print(my_code)
print(my_error_model)
print(my_decoder)

RotatedPlanarCode(7, 7)
DepolarizingErrorModel()
RotatedPlanarMPSDecoder(8, 'c', None)


In [5]:
code = RotatedPlanarCode(6,6)
print(code.site_bounds)
code_index=(0,0)
code.is_z_plaquette(code_index)
def _rotate_q_index(index, code):
    """Convert code site index in format (x, y) to tensor network q-node index in format (r, c)"""
    site_x, site_y = index  # qubit index in (x, y)
    site_r, site_c = code.site_bounds[1] - site_y, site_x  # qubit index in (r, c)
    print(site_r, site_c)
    return code.site_bounds[0] - site_c + site_r, site_r + site_c  # q-node index in (r, c)

_rotate_q_index(code_index, code)

(5, 5)
5 0


(10, 5)

In [6]:
tn_max_r, _ = _rotate_q_index((0, 0), code)
_, tn_max_c = _rotate_q_index((code.site_bounds[0], 0), code)
tn = np.empty((tn_max_r + 1, tn_max_c + 1), dtype=object)
tn.shape

5 0
5 5


(11, 11)

### Generate a random error

In [7]:
import collections
import itertools
import numpy as np
from qecsim import paulitools as pt
import matplotlib.pyplot as plt
import qecsim
from qecsim import app
from qecsim.models.generic import PhaseFlipErrorModel,DepolarizingErrorModel,BiasedDepolarizingErrorModel
from qecsim.models.planar import PlanarCode, PlanarMPSDecoder
from qecsim.models.rotatedplanar import RotatedPlanarCode, RotatedPlanarMPSDecoder

from _planarmpsdecoder_def import PlanarMPSDecoder_def
from _rotatedplanarmpsdecoder_def import RotatedPlanarMPSDecoder_def
import app_def
import importlib as imp
imp.reload(app_def)
import os, time
import multiprocessing as mp
from functools import partial
from itertools import cycle

In [11]:
# set physical error probability to 10%
error_probability = 0.4
bias =0.5
# seed random number generator for repeatability
rng = np.random.default_rng(10)
error_model = BiasedDepolarizingErrorModel(bias,'Z')
chi_val=10
decoder = RotatedPlanarMPSDecoder_def(chi=chi_val)
    
# error: random error based on error probability
error = error_model.generate(code, error_probability, rng)
qsu.print_pauli('error:\n{}'.format(code.new_pauli(error)))
code.site_bounds

(5, 5, 1, 1)

### Evaluate the syndrome
The syndrome is a binary array indicating the stabilizers with which the error does not commute. It is calculated as $syndrome = error \odot stabilisers^T$.

In [31]:
# syndrome: stabilizers that do not commute with the error
syndrome = pt.bsp(error, my_code.stabilizers.T)
qsu.print_pauli('syndrome:\n{}'.format(my_code.ascii_art(syndrome)))

### Find a recovery operation
In this case, the recovery operation is found by contracting a tensor network defined to have the value of the coset :

* Create a sample recovery operation by applying strings of Paulis between syndrome plaquettes and appropriate boundaries.
* Define tensor networks corresponding to the probability of each left coset of the generator group with the sample recovery and logical Pauli operations.
* Contract each tensor network (approximately) to evaluate the coset probabilities.
* Return any recovery from the most probable coset.

In [32]:
# recovery: best match recovery operation based on decoder
recovery = my_decoder.decode(my_code, syndrome)
qsu.print_pauli('recovery:\n{}'.format(my_code.new_pauli(recovery)))

As a sanity check, we expect $recovery \oplus error$ to commute with all stabilizers, i.e. $(recovery \oplus error) \odot stabilisers^T = 0$.

In [33]:
# check recovery ^ error commutes with stabilizers (by construction)
print(pt.bsp(recovery ^ error, my_code.stabilizers.T))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [1]:
x,y=2,3

### Visualise $recovery \oplus error$
Just out of curiosity, we can see what $recovery \oplus error$ looks like. If successful, it should be a product of stabilizer plaquette operators.

In [34]:
# print recovery ^ error (out of curiosity)
qsu.print_pauli('recovery ^ error:\n{}'.format(my_code.new_pauli(recovery ^ error)))

### Test if the recovery operation is successful
The recovery operation is successful iff $recovery \oplus error$ commutes with all logical operators, i.e. $(recovery \oplus error) \odot logicals^T = 0.$

In [35]:
# success iff recovery ^ error commutes with logicals
print(pt.bsp(recovery ^ error, my_code.logicals.T))

[0 0]


Note: The decoder is not guaranteed to find a successful recovery operation. The rotated planar 7 x 7 code has distance $d = 7$ so we can only guarantee to correct errors up to weight $(d - 1)/2=3$.

### Equivalent code in single call
The above demo is equivalent to the following code.

In [4]:
# repeat demo in single call
from qecsim import app
print(app.run_once(my_code, my_error_model, my_decoder, error_probability))

{'error_weight': 7, 'success': True, 'logical_commutations': array([0, 0]), 'custom_values': None}


In [40]:
%matplotlib inline
import collections
import itertools
import numpy as np
from qecsim import paulitools as pt
import matplotlib.pyplot as plt
from qecsim import app
from qecsim.models.generic import PhaseFlipErrorModel,DepolarizingErrorModel
from qecsim.models.planar import PlanarCode, PlanarMPSDecoder
from _planarmpsdecoder_def import PlanarMPSDecoder_def
import app_def
import importlib as imp
imp.reload(app_def)
sizes= range(5,9,2)
codes_and_size = [PlanarCode(*(size,size)) for size in sizes]

In [18]:
codes_and_size

[PlanarCode(5, 5), PlanarCode(7, 7)]

In [19]:
x=np.zeros(5)

In [20]:
x=np.append(x,1)

In [21]:
x

array([0., 0., 0., 0., 0., 1.])

In [23]:
for code_index in itertools.product(range(-1, 5 + 1), range(-1, 5 + 1)):
    print(code_index)

(-1, -1)
(-1, 0)
(-1, 1)
(-1, 2)
(-1, 3)
(-1, 4)
(-1, 5)
(0, -1)
(0, 0)
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(0, 5)
(1, -1)
(1, 0)
(1, 1)
(1, 2)
(1, 3)
(1, 4)
(1, 5)
(2, -1)
(2, 0)
(2, 1)
(2, 2)
(2, 3)
(2, 4)
(2, 5)
(3, -1)
(3, 0)
(3, 1)
(3, 2)
(3, 3)
(3, 4)
(3, 5)
(4, -1)
(4, 0)
(4, 1)
(4, 2)
(4, 3)
(4, 4)
(4, 5)
(5, -1)
(5, 0)
(5, 1)
(5, 2)
(5, 3)
(5, 4)
(5, 5)


In [ ]:
import collections
import itertools
import numpy as np
from qecsim import paulitools as pt
import matplotlib.pyplot as plt
import qecsim
from qecsim import app
from qecsim.models.generic import PhaseFlipErrorModel,DepolarizingErrorModel,BiasedDepolarizingErrorModel
from qecsim.models.planar import PlanarCode, PlanarMPSDecoder
from qecsim.models.rotatedplanar import RotatedPlanarCode, RotatedPlanarMPSDecoder

from _planarmpsdecoder_def import PlanarMPSDecoder_def
from _rotatedplanarmpsdecoder_def import RotatedPlanarMPSDecoder_def
import app_def
import importlib as imp
imp.reload(app_def)
import os, time
import multiprocessing as mp
from functools import partial
from itertools import cycle

def parallel_step_p(code,hadamard_mat,error_model, decoder, max_runs,error_probability):
    result= app_def.run_def(code,hadamard_mat, error_model, decoder, error_probability, max_runs)
    return result

# set models
sizes= range(5,10,4)
codes_and_size = [RotatedPlanarCode(*(size,size)) for size in sizes]
bias_list=[30,100000]

code_name="optimal"
code_name="random"
code_name="CSS"
code_name="XZZX"

if (code_name=="random"):
    realizations=50
else:
    realizations=1

# set physical error probabilities
error_probability_min, error_probability_max = 0.05, 0.24
error_probabilities = np.linspace(error_probability_min, error_probability_max, 10)
# set max_runs for each probability
max_runs = 2000

timestr = time.strftime("%Y%m%d-%H%M%S ")   #record current date and time
dirname="./data/"+timestr+code_name
os.mkdir(dirname)     

for bias in bias_list:
    error_model = BiasedDepolarizingErrorModel(bias,'Z')
    chi_val=10
    decoder = RotatedPlanarMPSDecoder_def(chi=chi_val)

    # print run parameters
    print('codes_and_size:', [code.label for code in codes_and_size])
    print('Error model:', error_model.label)
    print('Decoder:', decoder.label)
    print('Error probabilities:', error_probabilities)
    print('Maximum runs:', max_runs)

    pL_list_rand =np.zeros((len(codes_and_size),realizations,len(error_probabilities)))
    std_list_rand=np.zeros((len(codes_and_size),realizations,len(error_probabilities)))

    pL_list =np.zeros((len(codes_and_size),len(error_probabilities)))
    std_list=np.zeros((len(codes_and_size),len(error_probabilities)))
    def _rotate_q_index(index, code):
        """Convert code site index in format (x, y) to tensor network q-node index in format (r, c)"""
        site_x, site_y = index  # qubit index in (x, y)
        site_r, site_c = code.site_bounds[1] - site_y, site_x  # qubit index in (r, c)
        return code.site_bounds[0] - site_c + site_r, site_r + site_c  # q-node index in (r, c)

    for code_index,code in enumerate(codes_and_size):
        tn_max_r, _ = _rotate_q_index((0, 0), code)
        _, tn_max_c = _rotate_q_index((code.site_bounds[0], 0), code)

        hadamard_mat=np.zeros((tn_max_r,tn_max_c))

        if code_name=="random":
            pH=0.5  
            for realization_index in range(realizations):
                for row, col in np.ndindex(hadamard_mat.shape):
                    if ((row%2==0 and col%2==0) or (row%2==1 and col%2==1)):
                        if(np.random.rand(1,1))<pH:
                            hadamard_mat[row,col]=1

                p=mp.Pool()
                func=partial(parallel_step_p,code,hadamard_mat, error_model, decoder, max_runs)
                result=p.map(func, error_probabilities)
                #print(result)
                p.close()
                p.join()
                for i in range(len(result)):
                    pL_list_rand[code_index][realization_index][i]=result[i][0]
                    std_list_rand[code_index][realization_index][i]=result[i][1]
                
            pL_list[code_index] = np.sum(pL_list_rand[code_index],axis=0)/realizations
            std_list[code_index] = np.sum(std_list_rand[code_index],axis=0)/realizations**2

        else:
            if code_name=="XZZX":
                for row, col in np.ndindex(hadamard_mat.shape):
                    if row%2==0 and col%2==0:
                        hadamard_mat[row,col]=1

            if code_name=="optimal":
                d=hadamard_mat.shape[0]
                for row, col in np.ndindex(hadamard_mat.shape):
                    if (row ==0 and col in range(0,d,4)) or (row==d-1 and col in range(2,d-1,4)) or (row%2==1 and col%2==1):
                        hadamard_mat[row,col]=1

            p=mp.Pool()
            func=partial(parallel_step_p,code,hadamard_mat,error_model, decoder, max_runs)
            result=p.map(func, error_probabilities)
            print(result)
            p.close()
            p.join()
            
            for i in range(len(result)):
                pL_list[code_index][i]=result[i][0]   
                std_list[code_index][i]=result[i][1]

        
    plt.figure(figsize=(20,10))
    lines = ["-",":","--","-."]
    linecycler = cycle(lines)
    plt.title('TND at bias='+str(bias)+' and xi='+str(chi_val))
    for sizes_index,size in enumerate(sizes):
        plt.errorbar(error_probabilities,pL_list[sizes_index],std_list[sizes_index])
    plt.xlabel('p')
    plt.ylabel('$p_L$')
    plt.legend(sizes) 
    plt.savefig(dirname+"/threshold_plot_bias_"+str(bias)+".pdf")


In [456]:
rng = np.random.default_rng() if rng is None else rng
size=3
code=PlanarCode(*(size,size+1)) 

n_qubits =code.n_k_d[0]
probability=0.06
error_model = BiasedDepolarizingErrorModel(bias,'Z')
p=error_model.probability_distribution(probability)

p

(0.94, 0.000297029702970297, 0.000297029702970297, 0.0594059405940594)

In [457]:
#     3x4 planar code (H=qubit on horizontal edge, V=qubit on vertical edge):
#     ::
#       H1---H2---H3---H4
#           |   |   |
#           V13 V14 V15
#           |   |   |
#       H5---H6---H7---H8
#           |   |   |
#           V16 V17 V18
#           |   |   |
#       H9---H10--H11-H12


In [458]:
code.n_k_d

(18, 1, 3)

In [459]:
error_pauli = ''.join(rng.choice(
            ('I', 'X', 'Y', 'Z'),
            size=n_qubits,
            p=error_model.probability_distribution(probability)
        ))
error_pauli

'IIIIIIIIIZIIIIIZII'

In [465]:
error=pt.pauli_to_bsf(error_pauli)
error

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0])

In [471]:
len(error)

36

In [475]:
index = [i for i, x in enumerate(error) if x > 0]
index


[27, 33]

In [461]:
rng.choice(5, 3)

array([0, 2, 2])

In [462]:
syndrome = pt.bsp(error, code.stabilizers.T)
syndrome

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0])

In [63]:
code.stabilizers.T

array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [38]:
error.T

array([1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 0, 1, 1, 0, 1])

In [499]:
#    3x3 rotated planar code with H or V indicating qubits and hashed/blank plaquettes indicating X/Z stabilizers:
#     ::

#            /---\
#            |   |
#            H6--V7--H8\
#            |###|   |##|
#            |###|   |##|
#            |###|   |##|
#         /--V3---H4---V5-/
#         |##|   |###|
#         |##|   |###|
#         |##|   |###|
#         \--H0---V1---H2
#                |   |
#                \---/


             (0,2)-----(1,2)-----(2,2)
               |         |         |
               |         |         |
               |         |         |
             (0,1)-----(1,1)-----(2,1)
               |         |         |
               |         |         |
               |         |         |
             (0,0)-----(1,0)-----(2,0)

rng = np.random.default_rng() if rng is None else rng
size=3
code=RotatedPlanarCode(*(size,size)) 

n_qubits =code.n_k_d[0]
probability=0.1
error_model = BiasedDepolarizingErrorModel(bias,'Z')
p=error_model.probability_distribution(probability)

error_pauli = ''.join(rng.choice(
            ('I', 'X', 'Y', 'Z'),
            size=n_qubits,
            p=error_model.probability_distribution(probability)
        ))
error=pt.pauli_to_bsf(error_pauli)
syndrome = pt.bsp(error, code.stabilizers.T)
syndrome

array([0, 0, 0, 0, 0, 1, 0, 0])

In [505]:
print(error)
index = [i for i, x in enumerate(error) if x > 0]
index


[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]


[11]

In [501]:
code.stabilizers.T

array([[0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0]])

In [491]:
hvec=np.random.randint(2,size=(len(error_pauli),1))
hvec

array([[0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0]])

In [97]:
q=len(error_pauli)
for i in range(len(error_pauli)):
    if hvec[i]==1:
        error_temp=error[i]
        error[i]=error[q+i]
        error[q+i]=error_temp

In [37]:
error

NameError: name 'error' is not defined

In [81]:
hvec.T

array([[1, 0, 1, 1, 0, 1, 0, 0, 0]])

In [128]:
paulis = ('I', 'X', 'Y', 'Z')
bias=1000
error_model = BiasedDepolarizingErrorModel(bias,'Z')

prob_dist=error_model.probability_distribution(probability)
op_to_pr = dict(zip(paulis, prob_dist))
f='X'
f = pt.pauli_to_bsf(f)
I, X, Y, Z = pt.pauli_to_bsf(paulis)
n=0.1
e=0.1
s=0.1
w=0.1
# n, e, s, w are in {0, 1} so multiply op to turn on or off
op = (f + (n * Z) + (e * X) + (s * Z) + (w * X)) % 2
op

array([1.2, 0.2])

In [129]:
pt.bsf_to_pauli(op)

'X.6'

In [130]:
pt.bsf_to_pauli(np.array([1, 0]))

'X'

In [131]:
pt.pauli_to_bsf('X')

array([1, 0])

In [132]:
op_to_pr['X']

0.0003996003996003996

In [133]:
op_to_pr = dict(zip(paulis, prob_dist))
op_to_pr

{'I': 0.19999999999999996,
 'X': 0.0003996003996003996,
 'Y': 0.0003996003996003996,
 'Z': 0.7992007992007992}

In [134]:
n_qubits

9

In [136]:
hadamard_vec=np.zeros(n_qubits)
for i in range(n_qubits):
    if i%2:
        hadamard_vec[i]=1

In [137]:
hadamard_vec

array([0., 1., 0., 1., 0., 1., 0., 1., 0.])

In [3]:
import numpy as np
np.random.rand(1,1)

array([[0.55376431]])

In [19]:
count =0
for i in range(1000000):
    x=np.random.rand(1,1)
    if x<=0.5:
        count+=1
count

500239

In [23]:
hadamard_mat=np.zeros((5,5))
np.prod(hadamard_mat.shape)

25

In [27]:
from qecsim.models.rotatedplanar import RotatedPlanarCode, RotatedPlanarMPSDecoder

size=3
code=RotatedPlanarCode(*(size,size)) 
tn_max_r, _ = _rotate_q_index((0, 0), code)
_, tn_max_c = _rotate_q_index((code.site_bounds[0], 0), code)

print(tn_max_r,tn_max_c)

4 4


In [26]:

            def _rotate_q_index(index, code):
                """Convert code site index in format (x, y) to tensor network q-node index in format (r, c)"""
                site_x, site_y = index  # qubit index in (x, y)
                site_r, site_c = code.site_bounds[1] - site_y, site_x  # qubit index in (r, c)
                return code.site_bounds[0] - site_c + site_r, site_r + site_c  # q-node index in (r, c)

In [29]:
code._plaquette_indices

[(1, -1), (0, 0), (1, 1), (0, 2), (-1, 0), (1, 0), (0, 1), (2, 1)]

In [30]:
code.site_bounds

(2, 2)

In [36]:
code.is_z_plaquette((2,0))

True

In [347]:
#    3x3 rotated planar code with H or V indicating qubits and hashed/blank plaquettes indicating X/Z stabilizers:
#     ::

#            /---\
#            |   |
#            H6--V7--H8\
#            |###|   |##|
#            |###|   |##|
#            |###|   |##|
#         /--V3---H4---V5-/
#         |##|   |###|
#         |##|   |###|
#         |##|   |###|
#         \--H0---V1---H2
#                |   |
#                \---/

import collections
import itertools
import numpy as np
from qecsim import paulitools as pt
import matplotlib.pyplot as plt
import qecsim
from qecsim import app
from qecsim.models.generic import PhaseFlipErrorModel,DepolarizingErrorModel,BiasedDepolarizingErrorModel
#from qecsim.models.planar import PlanarCode, PlanarMPSDecoder
from qecsim.models.rotatedplanar import RotatedPlanarCode

from _planarmpsdecoder_def import PlanarMPSDecoder_def
from _rotatedplanarmpsdecoder_def import RotatedPlanarMPSDecoder_def
import app_def
import _rotatedplanarmpsdecoder_def
import importlib as imp
imp.reload(app_def)
imp.reload(_rotatedplanarmpsdecoder_def)
import os, time
import multiprocessing as mp
from functools import partial

import collections
import itertools
import json
import logging
import statistics
import time
import numpy as np
from qecsim import paulitools as pt
from qecsim.error import QecsimError
from qecsim.model import DecodeResult

rng = np.random.default_rng(10)

size=3
code=RotatedPlanarCode(*(size,size)) 
bias=100
n_qubits =code.n_k_d[0]
probability=0.2
error_model = BiasedDepolarizingErrorModel(bias,'Z')
p=error_model.probability_distribution(probability)

error_pauli = ''.join(rng.choice(
            ('I', 'X', 'Y', 'Z'),
            size=n_qubits,
            p=error_model.probability_distribution(probability)
        ))
error=pt.pauli_to_bsf(error_pauli)
syndrome = pt.bsp(error, code.stabilizers.T)
syndrome

array([0, 0, 0, 0, 1, 1, 1, 0])

In [348]:
code.stabilizers.T


array([[0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0]])

In [349]:
error_pauli

'ZIZIIIIZI'

In [98]:
code.site_bounds

(2, 2)

In [99]:
int(2*0.6)

1

In [100]:
pH=0.5

In [356]:

size=3
code=RotatedPlanarCode(*(size,size+1)) 

hadamard_mat=np.zeros((3,4))
from sklearn.utils.random import sample_without_replacement

def random_coords(dims, nsamp):
    idx = sample_without_replacement(np.prod(dims), nsamp)
    return np.vstack(np.unravel_index(idx, dims)).T

rand_coords= random_coords(hadamard_mat.shape,int(np.prod(hadamard_mat.shape)*pH))
for row, col in rand_coords:
    hadamard_mat[row,col]=1

In [357]:
hadamard_mat

array([[0., 1., 0., 0.],
       [0., 1., 1., 1.],
       [1., 0., 0., 1.]])

In [360]:
n_qubits = code.n_k_d[0]
hadamard_vec=np.zeros(n_qubits)

Nx=code.site_bounds[0]+1
Ny=code.site_bounds[1]+1

for i,j in np.ndindex(hadamard_mat.shape):
    if hadamard_mat[i,j]==1:
        hadamard_vec[i+j*(Nx-1)]=1
        
hadamard_vec

array([0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1.])

In [118]:
code=RotatedPlanarCode(*(size,size+1)) 
n_qubits =code.n_k_d[0]
hadamard_vec=np.zeros(n_qubits)

hadamard_mat=np.zeros((3,4))
for row, col in np.ndindex(hadamard_mat.shape):
    if (row+col)%2==0:
        hadamard_mat[row,col]=1
for i in range(n_qubits):
    if i%2==0:
        hadamard_vec[i]=1

In [119]:
hadamard_mat

array([[1., 0., 1., 0.],
       [0., 1., 0., 1.],
       [1., 0., 1., 0.]])

In [120]:
hadamard_vec

array([1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.])

In [123]:
hadamard_mat=np.zeros((3,4))
rand_coords= random_coords(hadamard_mat.shape,int(np.prod(hadamard_mat.shape)*pH))
for row, col in rand_coords:
        hadamard_mat[row,col]=1

In [124]:
hadamard_mat

array([[0., 0., 1., 1.],
       [1., 0., 1., 0.],
       [1., 1., 0., 0.]])

In [125]:
rand_coords

array([[2, 0],
       [0, 3],
       [1, 0],
       [2, 1],
       [0, 2],
       [1, 2]])

In [126]:
dims=(3,2)
np.prod(dims)

6

In [130]:
y=random_coords((3,3),4)

In [129]:
x=random_coords((3,3),9)

In [137]:
(3,4) in x

False

In [139]:
error_pauli = ''.join(rng.choice(
        ('I', 'X', 'Y', 'Z'),
        size=n_qubits,
        p=error_model.probability_distribution(probability)
    ))

In [140]:
error_pauli

'ZZIIIZZIIIZI'

In [147]:
x=['X','Y','Z']
x

['X', 'Y', 'Z']

In [149]:
''.join(x)

'XYZ'

In [180]:
x=random_coords(10,2)
y=random_coords(10,10)

x

array([[6],
       [4]])

In [181]:
y

array([[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7],
       [8],
       [9]])

In [182]:
[5] in x

False

In [192]:
def complement(listA, listB):
    listC=[]    
    for i in range(len(listA)):
        if listA[i] not in listB:
            listC.append(listA[i][0])
    return listC

In [193]:
z=complement(y,x)

In [194]:
z

[0, 1, 2, 3, 5, 7, 8, 9]

In [212]:
from random import shuffle
x = ['X','Y','Z','I','X','Y','Z']
shuffle(x)
''.join(x)

'ZYIXYZX'

In [199]:
x

[0, 4, 1, 3, 8, 9, 2, 5, 6, 7]

In [208]:
pI=0.2;p=(1-pI)/3
y=[]

In [209]:
for i in range(int(n_qubits*pI)):
    y.append('I')
        
for i in range(int(n_qubits*p)):
    y.append('X')

for i in range(int(n_qubits*p)):
    y.append('Y')
for i in range(int(n_qubits*p)):
    y.append('Z')
  
  
y
   

['I', 'I', 'X', 'X', 'X', 'Y', 'Y', 'Y', 'Z', 'Z', 'Z']

In [210]:
shuffle(y)
y

['X', 'X', 'Z', 'I', 'I', 'Y', 'Z', 'X', 'Z', 'Y', 'Y']

In [245]:
from random import shuffle
def generate(error_model, code, probability, rng=None):
    """
    See :meth:`qecsim.model.ErrorModel.generate`

    Notes:

    * This method delegates to :meth:`probability_distribution` to find the probability of I, X, Y, Z operators on
      each qubit, assuming an IID error model.
    """
    n_qubits = code.n_k_d[0]
    (pI,pX,pY,pZ)=error_model.probability_distribution(probability)
    error_Pauli=[]
    error_Pauli.extend('I'*int(n_qubits*pI))    
    error_Pauli.extend('X'*int(n_qubits*pX))    
    error_Pauli.extend('Y'*int(n_qubits*pY))    
    error_Pauli.extend('Z'*int(n_qubits*pZ))    
    shuffle(error_Pauli)
    print(error_Pauli)
    return pt.pauli_to_bsf(''.join(error_Pauli))

In [246]:
error_Pauli=generate(error_model,code,0.6)
error_Pauli

['I', 'Z', 'I', 'I', 'I', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z']


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1])

In [285]:
code=RotatedPlanarCode(*(5,5)) 

error_model = BiasedDepolarizingErrorModel(0.5,'Z')

n_qubits = code.n_k_d[0]
(pI,pX,pY,pZ)=error_model.probability_distribution(0.8)
error_Pauli=[]
error_Pauli.extend('I'*round(n_qubits*pI))    
error_Pauli.extend('X'*round(n_qubits*pX))    
error_Pauli.extend('Y'*round(n_qubits*pY))    
error_Pauli.extend('Z'*round(n_qubits*pZ))    
error_Pauli.extend('I'*(n_qubits-round(n_qubits*pI)-round(n_qubits*pX)-round(n_qubits*pY)-round(n_qubits*pZ)))
print(error_Pauli)
for i in range(n_qubits):
    if hadamard_vec[i]==1:
        if error_Pauli[i]=='X':
            error_Pauli[i]='Z'
        elif error_Pauli[i]=='Z':
            error_Pauli[i]='X'
            
print(error_Pauli)


['I', 'I', 'I', 'I', 'I', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z']
['I', 'I', 'I', 'I', 'I', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'X', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Z', 'X', 'X', 'X', 'X', 'X', 'Z']


In [283]:
hadamard_vec=np.random.randint(0,2,n_qubits)

In [284]:
hadamard_vec

array([0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1])

In [275]:
error_Pauli[3]='Z'

In [276]:
error_Pauli

['I', 'I', 'X', 'Z', 'X', 'Y', 'Y', 'Y', 'Z', 'Z', 'Z', 'I']

In [299]:
n_qubits = code.n_k_d[0]
(pI,pX,pY,pZ)=error_model.probability_distribution(0.8)
error_Pauli=[]
error_Pauli.extend('X'*round(n_qubits*pX))    
error_Pauli.extend('Y'*round(n_qubits*pY))    
error_Pauli.extend('Z'*round(n_qubits*pZ))    
error_Pauli.extend('I'*(n_qubits-len(error_Pauli)))
len(error_Pauli)== n_qubits

True

In [297]:
len(error_Pauli)

27

In [295]:
n_qubits

25

In [296]:
error_Pauli.extend('I'*-1)


In [311]:
error_probability_min, error_probability_max = 0.05, 0.5
error_probabilities = np.linspace(error_probability_min, error_probability_max, 18)

In [312]:
sizes

range(5, 9, 2)

In [346]:
def removedup(vec):
    vec2 = [] 
    [vec2.append(x) for x in vec if x not in vec2] 
    return np.array(vec2)

n_qubits=sizes[0]**2
vround = np.vectorize(round)
n_error_vec=removedup(vround(error_probabilities*n_qubits))

n_errors =[]
n_errors.extend(n_error_vec)

for i in range(1,len(sizes)):
    n_qubits=sizes[i]**2
    n_error_vec=removedup(vround(error_probabilities*n_qubits))
    n_errors.extend(n_error_vec[n_error_vec>n_errors[-1]])
    
n_errors

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 21, 22, 23, 24]

In [342]:
n_error_vec[n_error_vec>9]

TypeError: '>' not supported between instances of 'list' and 'int'

In [319]:
round(1.5)

2

In [320]:
vfunc = np.vectorize(round)
vfunc([0.3,2.5])

array([0, 2])

In [322]:
error_probabilities*sizes[0]

array([0.25      , 0.38235294, 0.51470588, 0.64705882, 0.77941176,
       0.91176471, 1.04411765, 1.17647059, 1.30882353, 1.44117647,
       1.57352941, 1.70588235, 1.83823529, 1.97058824, 2.10294118,
       2.23529412, 2.36764706, 2.5       ])

In [324]:
n_errors =[]
vround = np.vectorize(round)

n_qubits=sizes[0]**2
n_errors.extend(vround(error_probabilities*n_qubits))

In [325]:
n_errors

[1, 2, 3, 3, 4, 5, 5, 6, 7, 7, 8, 9, 9, 10, 11, 11, 12, 12]

In [361]:
max_site_x, max_site_y = code.site_bounds
for code_index in itertools.product(range(-1, max_site_x + 1), range(-1, max_site_y + 1)):
    print(code_index)

(-1, -1)
(-1, 0)
(-1, 1)
(-1, 2)
(0, -1)
(0, 0)
(0, 1)
(0, 2)
(1, -1)
(1, 0)
(1, 1)
(1, 2)
(2, -1)
(2, 0)
(2, 1)
(2, 2)
(3, -1)
(3, 0)
(3, 1)
(3, 2)


In [378]:
code=PlanarCode(*(5,5)) 
rng = np.random.default_rng(59)
bias=100

error_model = BiasedDepolarizingErrorModel(bias,'Z')
chi_val=10
decoder = PlanarMPSDecoder_def(chi=chi_val)
error = error_model.generate(code, error_probability_max, rng)
syndrome = pt.bsp(error, code.stabilizers.T)
sample_pauli = decoder.sample_recovery(code, syndrome)
hadamard_mat_sample=np.zeros((2*sample_pauli.code.size[0] - 1, 2*sample_pauli.code.size[1] - 1))
hadamard_mat=np.zeros(hadamard_mat_sample.shape)

n_qubits =code.n_k_d[0]

        
for row, col in np.ndindex(hadamard_mat.shape):
    if row%2==0 and col%2==0:
        hadamard_mat[row,col]=1
# for i in range(n_qubits):
#     if i<sizes[code_index]**2
#         hadamard_vec[i]=1
n_qubits = code.n_k_d[0]
hadamard_vec=np.zeros(np.prod(hadamard_mat.shape))

for i,j in np.ndindex(hadamard_mat.shape):
    if hadamard_mat[i,j]==1:
        hadamard_vec[j+i*hadamard_mat.shape[1]]=1
        
hadamard_vec2=[]        
for row, col in np.ndindex(hadamard_mat.shape):
    if (row%2==0 and col%2==0):
        hadamard_mat[row,col]=1
        hadamard_vec2.append(hadamard_mat[row,col])

for row, col in np.ndindex(hadamard_mat.shape):
    if (row%2==1 and col%2==1):
        hadamard_vec2.append(hadamard_mat[row,col])
        
        
hadamard_vec2=np.array(hadamard_vec2)        

In [379]:
hadamard_vec

array([1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.])

In [380]:
hadamard_vec2

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.])

In [512]:
code=RotatedPlanarCode(*(5,5)) 
rng = np.random.default_rng(59)
bias=100
sizes=[5]
error_model = BiasedDepolarizingErrorModel(bias,'Z')
chi_val=10
hadamard_mat=np.zeros((sizes[0],sizes[0]))

n_qubits =code.n_k_d[0]
       
for row, col in np.ndindex(hadamard_mat.shape):
    if (row+col)%2==0:
        hadamard_mat[row,col]=1
        
# for i in range(n_qubits):
#     if i<sizes[code_index]**2
#         hadamard_vec[i]=1
n_qubits = code.n_k_d[0]
hadamard_vec=np.zeros(np.prod(hadamard_mat.shape))

for i,j in np.ndindex(hadamard_mat.shape):
    if hadamard_mat[i,j]==1:
        hadamard_vec[j+i*hadamard_mat.shape[1]]=1

In [394]:
hadamard_vec[[1,2,6]]=0

In [513]:
hadamard_vec

array([1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 1.])

In [398]:
hadamard_vec[[1,2,3,4,5,6,8]]=0.3423
hadamard_vec


array([1.    , 0.3423, 0.3423, 0.3423, 0.3423, 0.3423, 0.3423, 0.3423,
       0.3423, 0.3423, 1.    , 0.    , 1.    , 0.    , 1.    , 0.    ,
       1.    , 0.    , 1.    , 0.    , 1.    , 0.    , 1.    , 0.    ,
       1.    ])

In [401]:
hadamard_vec[range(1,5)]=0.3423
hadamard_mat= np.zeros((5,5))
hadamard_mat[1,range(0,3)]=0.342
hadamard_mat

array([[0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.342, 0.342, 0.342, 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   ]])

In [414]:
hadamard_mat= np.zeros((7,7))

dy=hadamard_mat.shape[0]
dx=hadamard_mat.shape[1]
for row, col in np.ndindex(hadamard_mat.shape):
    if row%2==0:
        if row%4==0:
            hadamard_mat[row,range(0,dx-1)]=1
        else:
            hadamard_mat[row,range(1,dx)]=1
    else:
        if row%4==1:
            hadamard_mat[row,dx-1]=1
        elif row%4==3:
            hadamard_mat[row,0]=1


In [415]:
hadamard_mat

array([[1., 1., 1., 1., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 1., 1., 1., 1., 1., 1.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 1., 1., 1., 1., 1., 1.]])

In [423]:
res=[1,2,3]
def f(a):
    return a**2

fv=np.vectorize(f)

fv(res)

array([1, 4, 9])

In [432]:
x=1
x+=2%3

In [433]:
x

3

In [434]:
pt.pauli_to_bsf('X')

array([1, 0])

In [435]:
y,z=x

TypeError: cannot unpack non-iterable int object

In [518]:
error_probability=0.2
step_error = error_model.generate(code, error_probability, rng)
print(step_error)
print(np.vectorize(round)(hadamard_vec))
for i in range(n_qubits):
    if hadamard_vec[i]==1:
        step_error_temp=step_error[i]
        step_error[i]=step_error[n_qubits+i]
        step_error[n_qubits+i]=step_error_temp
        
print(step_error)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 1 1 0
 0 0 0 0 0 0 0 1 0 0 0 1 0]
[1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0
 0 0 0 0 0 0 0 1 0 0 0 1 0]


In [508]:
hadamard_vec

array([1.    , 0.3423, 0.3423, 0.3423, 0.3423, 0.3423, 0.3423, 0.3423,
       0.3423, 0.3423, 1.    , 0.    , 1.    , 0.    , 1.    , 0.    ,
       1.    , 0.    , 1.    , 0.    , 1.    , 0.    , 1.    , 0.    ,
       1.    ])

In [510]:
print(step_error)

[0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0]


In [526]:
hadamard_mat.shape

(5, 5)

In [523]:
hadamard_mat[0,1]=1

In [524]:
hadamard_vec=np.zeros(np.prod(hadamard_mat.shape))
for i,j in np.ndindex(hadamard_mat.shape):
    hadamard_vec[(i+j*hadamard_mat.shape[1])]=hadamard_mat[i,j]

In [525]:
hadamard_vec

array([1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 1.])

In [521]:
hadamard_vec2=np.zeros(np.prod(hadamard_mat.shape))
for i in range(np.prod(hadamard_mat.shape)):
    if i%2==0:
        hadamard_vec2[i]=1
hadamard_vec2

array([1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 1.])

In [522]:
hadamard_vec2==hadamard_vec

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [534]:
x=np.ndindex(hadamard_mat.shape)

In [535]:
for i,j in x:
    print(j,i)

0 0
1 0
2 0
3 0
4 0
0 1
1 1
2 1
3 1
4 1
0 2
1 2
2 2
3 2
4 2
0 3
1 3
2 3
3 3
4 3
0 4
1 4
2 4
3 4
4 4


In [537]:
nrows, ncols=hadamard_mat.shape

In [538]:
nrows,ncols

(5, 5)

In [539]:
np.ndindex(5,5)

In [541]:
np.prod((nrows,ncols))

25

ValueError: not enough values to unpack (expected 3, got 2)

In [566]:
import numpy as np
import random
import scipy
import networkx as nx
import pymatching
from pymatching import Matching
from pymatching import set_seed
from scipy.sparse import coo_matrix
import matplotlib.pyplot as plt
import time
from itertools import cycle
import pickle
import multiprocessing as mp
np.seterr(divide='ignore')
from functools import partial
import qecsim
from qecsim import app
from qecsim.models.generic import PhaseFlipErrorModel,DepolarizingErrorModel,BiasedDepolarizingErrorModel

def dic_pbc(Nx,Ny):

    vX=np.zeros((Nx,Ny))
    for i in range(0,Nx):
        for j in range(0,Ny):
            vX[i,j]=i+j*Nx
    vZ=vX 
    return [vX,vZ]

def dic_obc(Nx,Ny):

    vX=np.zeros((Nx+2,Ny+2),dtype=int)
    virX1,virX2=(Nx+1)*(Ny-1),(Nx+1)*(Ny-1)+1

    for i in range(0,Nx+2):
        for j in range(0,Ny+2):
            if j==0:
                vX[i,j]=virX1
            elif j==Ny:
                vX[i,j]=virX2
            else:
                vX[i,j]=round(i-1+(j-1)*(Nx+1))

    vZ=np.zeros((Nx+2,Ny+2),dtype=int)
    virZ1,virZ2=Nx*Ny,Nx*Ny+1

    for i in range(0,Nx+2):
        for j in range(1,Ny+1):
            if i==0:
                vZ[i,j]=virZ1
            elif i==Nx+1:
                vZ[i,j]=virZ2
            else:
                vZ[i,j]=round(i-1+(j-1)*Nx)
    return [vX,vZ]

def graph_temp(Nx,Ny,prob_dist,periodic):
  
    pI,pX,pY,pZ=prob_dist
    #weights
    wZ=abs(np.log(pZ/pI))
    wX=abs(np.log(pX/pI))

    Graph_Zerr=nx.Graph()

    id=0
    if(periodic==1):
        [vX,vZ]=dic_pbc(Nx,Ny)
        for i in range(0,Nx):
            for j in range(0,Ny):
                Graph_Zerr.add_edge(vZ[i][j],vZ[(i + 1)%Nx][j],qubit_id=id,weight=wZ,error_probability=pZ)
                id+=1

                Graph_Zerr.add_edge(vZ[i][j],vZ[i][(j + 1)%Ny],qubit_id=id,weight=wZ,error_probability=pZ)
                id+=1
    else:
        virZ1,virZ2=Nx*Ny,Nx*Ny+1
        [vX,vZ]=dic_obc(Nx,Ny)
        for i in range(1,Nx+1):
            for j in range(1,Ny+1):
                if j==Ny:
                    Graph_Zerr.add_edge(vZ[i,j],vZ[i+1,j],qubit_id=id,weight=wZ,error_probability=pZ)  
                    id+=1
                else:
                    Graph_Zerr.add_edge(vZ[i,j],vZ[i+1,j],qubit_id=id,weight=wZ,error_probability=pZ)  
                    id+=1
                    Graph_Zerr.add_edge(vZ[i,j],vZ[i,j+1],qubit_id=id,weight=wZ,error_probability=pZ) 
                    id+=1
                if i==1:
                    Graph_Zerr.add_edge(vZ[i-1,j],vZ[i,j],qubit_id=id,weight=wZ,error_probability=pZ)  
                    id+=1

        Graph_Zerr.add_edge(virZ1,virZ2,qubit_id=-1,weight=0,error_probability=0)    

    return Graph_Zerr


def construct_lattice(Nx,Ny,prob_dist,periodic):

    pI,pX,pY,pZ=prob_dist
    #weights
    wZ=abs(np.log(pZ/pI))
    wX=abs(np.log(pX/pI))

    if periodic==0:
        virZ1,virZ2=Nx*Ny,Nx*Ny+1
        virX1,virX2=(Nx+1)*(Ny-1),(Nx+1)*(Ny-1)+1
        [vX,vZ]=dic_obc(Nx,Ny)

        Graph_Zerr=nx.Graph()
        Graph_Xerr=nx.Graph()

        X_logical1_Graph_Zerr=np.array([],dtype=int)
        Z_logical1_Graph_Xerr=np.array([],dtype=int)

        X_syndrome_num=Nx*Ny+2  #total number of X syndromes including the virtual ones 
        Z_syndrome_num=(Nx+1)*(Ny-1)+2  #total number of Z syndromes including the virtual ones 
        total_syndrome_num=X_syndrome_num+Z_syndrome_num

        #construct the Z lattice
        id=0
        for i in range(1,Nx+1):
            for j in range(1,Ny+1):
                if j==Ny:
                    Graph_Zerr.add_edge(vZ[i,j],vZ[i+1,j],qubit_id=id,weight=wZ,error_probability=pZ)  
                    Graph_Xerr.add_edge(vX[i+1,j-1],vX[i+1,j],qubit_id=id,weight=wX,error_probability=pX) 
                    id+=1
                else:
                    Graph_Zerr.add_edge(vZ[i,j],vZ[i+1,j],qubit_id=id,weight=wZ,error_probability=pZ)  
                    Graph_Xerr.add_edge(vX[i+1,j-1],vX[i+1,j],qubit_id=id,weight=wX,error_probability=pX) 
                    id+=1
                    Graph_Zerr.add_edge(vZ[i,j],vZ[i,j+1],qubit_id=id,weight=wZ,error_probability=pZ) 
                    Graph_Xerr.add_edge(vX[i,j],vX[i+1,j],qubit_id=id,weight=wX,error_probability=pX)
                    id+=1
                if i==1:
                    Graph_Zerr.add_edge(vZ[i-1,j],vZ[i,j],qubit_id=id,weight=wZ,error_probability=pZ)  
                    Graph_Xerr.add_edge(vX[i,j-1],vX[i,j],qubit_id=id,weight=wX,error_probability=pX) 
                    id+=1

        Graph_Zerr.add_edge(virZ1,virZ2,qubit_id=-1,weight=0,error_probability=0)  
        Graph_Xerr.add_edge(virX1,virX2,qubit_id=-1,weight=0,error_probability=0) 

        Z_logical_qubit_num=Nx+1  
        X_logical_qubit_num=Ny    

        for i in range(1,Nx+1):
            for j in range(1,Ny+1):
                if vZ[i,j]in range(0,Nx*Ny,Nx):
                    X_logical1_Graph_Zerr=np.append(X_logical1_Graph_Zerr,Graph_Zerr[vZ[i,j]][virZ1]['qubit_id'])

        for i in range(1,Nx+2):
            for j in range(1,Ny):
                if vX[i,j]in range(0,Nx+1):
                    Z_logical1_Graph_Xerr=np.append(Z_logical1_Graph_Xerr,Graph_Xerr[vX[i,j]][virX1]['qubit_id'])

        Graph_Zerr.nodes[virZ1]['is_boundary']=True
        Graph_Zerr.nodes[virZ2]['is_boundary']=True
        Graph_Xerr.nodes[virX1]['is_boundary']=True   
        Graph_Xerr.nodes[virX2]['is_boundary']=True   

        return [Graph_Xerr,Graph_Zerr,X_logical1_Graph_Zerr,Z_logical1_Graph_Xerr]

    else: 
        [vX,vZ]=dic_pbc(Nx,Ny)

        Graph_Zerr=nx.Graph()
        Graph_Xerr=nx.Graph()

        X_logical1_Graph_Zerr=np.array([],dtype=int)
        X_logical2_Graph_Zerr=np.array([],dtype=int)
        Z_logical1_Graph_Xerr=np.array([],dtype=int)
        Z_logical2_Graph_Xerr=np.array([],dtype=int)

        id=0
        for i in range(0,Nx):
            for j in range(0,Ny):
                Graph_Zerr.add_edge(vZ[i][j],vZ[(i + 1)% Nx][j],qubit_id=id,weight=wZ,error_probability=pZ)
                Graph_Xerr.add_edge(vX[(i+1)%Nx][(j-1)%Ny],vX[(i+1)%Nx][j],qubit_id=id,weight=wX,error_probability=pX)
                id+=1
                Graph_Zerr.add_edge(vZ[i][j],vZ[i][(j + 1)%Ny],qubit_id=id,weight=wZ,error_probability=pZ)
                Graph_Xerr.add_edge(vX[i][j],vX[(i + 1)% Nx][j],qubit_id=id,weight=wX,error_probability=pX)
                id+=1

        for i in range(0,Nx):
            for j in range(0,Ny):
                if vZ[i][j] in range(0,Nx*Ny,Nx):
                    X_logical1_Graph_Zerr=np.append(X_logical1_Graph_Zerr,Graph_Zerr[vZ[i][j]][vZ[(i + 1)% Nx][j]]['qubit_id'])
                if vX[i][j] in range(0,Nx):
                    Z_logical1_Graph_Xerr=np.append(Z_logical1_Graph_Xerr,Graph_Xerr[vX[i][j]][vX[i][(j+1)%Ny]]['qubit_id'])
                if vZ[i][j] in range(0,Nx):
                    X_logical2_Graph_Zerr=np.append(X_logical2_Graph_Zerr,Graph_Zerr[vZ[i][j]][vZ[i][(j+1)%Ny]]['qubit_id'])
                if vX[i][j] in range(0,Nx*Ny,Nx):
                    Z_logical2_Graph_Xerr=np.append(Z_logical2_Graph_Xerr,Graph_Xerr[vX[i][j]][vX[(i + 1)% Nx][j]]['qubit_id'])

        return [Graph_Xerr,Graph_Zerr,X_logical1_Graph_Zerr,X_logical2_Graph_Zerr,Z_logical1_Graph_Xerr,Z_logical2_Graph_Xerr]


def weight_error_changes(prob_dist,Graph_Xerr,Graph_Zerr,periodic,hadamard_vec):
  
    pI,pX,pY,pZ=prob_dist
    #weights
    wZ=abs(np.log(pZ/pI))
    wX=abs(np.log(pX/pI))
    
    if periodic==1:
        nqubits=len(Graph_Zerr.edges)
    else:
        nqubits=len(Graph_Zerr.edges)-1

    for ind in range(nqubits):
        if (hadamard_vec[ind]==1):
            for (u,v) in list(Graph_Zerr.edges):
                if Graph_Zerr[u][v]['qubit_id']==ind:
                    Graph_Zerr[u][v]['weight']=wX
                    Graph_Zerr[u][v]['error_probability']=pX
            for (u,v) in list(Graph_Xerr.edges):
                if Graph_Xerr[u][v]['qubit_id']==ind:
                    Graph_Xerr[u][v]['weight']=wZ
                    Graph_Xerr[u][v]['error_probability']=pZ

    return [Graph_Xerr,Graph_Zerr]


def spiral_torus(Nx,pitch):    
    Ny=Nx
    [vX,vZ]=dic_pbc(Nx,Ny)

    L_left=int_([Nx-1])
    pitch_left=int_([pitch])
    xcoord_pitch=int_([np.floor(L_left[-1]/pitch_left[-1])])

    while (pitch_left[-1]>1):
        L_left=np.append(L_left,L_left[-1]-int(np.floor(L_left[-1]/pitch_left[-1]))-1)
        pitch_left=np.append(pitch_left,pitch_left[-1]-1)
        xcoord_pitch=np.append(xcoord_pitch,xcoord_pitch[-1]+int(np.floor(L_left[-1]/pitch_left[-1]))+1)
    spiral=np.zeros((2*pitch*Nx,2))
    pitch_count=0
    
    for ist_temp in xcoord_pitch:
        ist=ist_temp
        jst=0
        for ind in range(2*Ny):
            if ind%2==0:
                spiral[ind+pitch_count*2*Ny]=[vZ[ist][jst],vZ[(ist+1)%Nx][jst]]
                ist=(ist+1)%Nx
            else:
                spiral[ind+pitch_count*2*Ny]=[vZ[ist][jst],vZ[ist][(jst+1)%Ny]]
                jst=(jst+1)%Ny
        pitch_count=pitch_count+1

    return spiral 

def func_hadamard_vec(Nx,Ny,pH,prob_dist,periodic):

    Graph_Zerr=graph_temp(Nx,Ny,prob_dist,periodic)
    if periodic==1:
        hadamard_vec=np.zeros((len(Graph_Zerr.edges)))
    else:
        hadamard_vec=np.zeros((len(Graph_Zerr.edges)-1))

    if pH==1.5:
        if periodic==1:
            for (u,v) in list(Graph_Zerr.edges):
                if abs(u-v)==1 or abs(u-v)==Nx-1: # horizontal edges
                    hadamard_vec[Graph_Zerr[u][v]['qubit_id']]=1
        else:
            for (u,v) in list(Graph_Zerr.edges):
                if abs(u-v)==Nx: #vertical edges: easier condition in case of surface code
                    if Graph_Zerr[u][v]['qubit_id']>-1:
                        hadamard_vec[Graph_Zerr[u][v]['qubit_id']]=1

    elif pH==2.5:        
        for (u,v) in list(Graph_Zerr.edges):
            if abs(u-v)==1 or abs(u-v)==Nx-1:
                if Graph_Zerr[u][v]['qubit_id']>-1:
                    hadamard_vec[Graph_Zerr[u][v]['qubit_id']]=1
        
        spacefillcurve_extra_hadamard=spiral_torus(Nx,pitch)

        for (u,v) in spacefillcurve_extra_hadamard:
            if Graph_Zerr[u][v]['qubit_id']>-1:
                hadamard_vec[Graph_Zerr[u][v]['qubit_id']]=(hadamard_vec[Graph_Zerr[u][v]['qubit_id']]+1)%2 

    elif (pH!=0 and pH!=1.5 and pH!=2.5):
        for i in range(0,len(hadamard_vec)):
            if(np.random.rand(1,1))<pH:
                hadamard_vec[i]=1

    return hadamard_vec

def parallel_step_p(bias,bias_str,error_model,Nx,Ny,num_mc,pH,hadamard_vec,num_realiz,periodic,error_probability): 
    def square(a):
        return a**2

    vsquare=np.vectorize(square)

    prob_dist= error_model.probability_distribution(error_probability)
 
    failure_list=np.zeros(num_mc) 

    result=construct_lattice(Nx,Ny,prob_dist,periodic)

    if periodic==0:
        [Graph_Xerr,Graph_Zerr,X_logical1_Graph_Zerr,Z_logical1_Graph_Xerr]=result
    else:
        [Graph_Xerr,Graph_Zerr,X_logical1_Graph_Zerr,X_logical2_Graph_Zerr,Z_logical1_Graph_Xerr,Z_logical2_Graph_Xerr]=result

    pL_rand=np.zeros(num_realiz)
    std_rand=np.zeros(num_realiz)    

    for realization_index in range(num_realiz):
        if num_realiz>1:
            hadamard_vec=func_hadamard_vec(Nx,Ny,pH,prob_dist,periodic)
        hadamard_vec=func_hadamard_vec(Nx,Ny,pH,prob_dist,periodic)
        print(hadamard_vec)
        if pH!=0:            
            [Graph_Xerr,Graph_Zerr]=weight_error_changes(prob_dist,Graph_Xerr,Graph_Zerr,periodic,hadamard_vec)

        # for i,j in Graph_Zerr.edges:
        #     print(i,j,Graph_Zerr[i][j]['weight'])
    
        mXerr=Matching(Graph_Xerr)
        mZerr=Matching(Graph_Zerr)

        for i in range(num_mc):
            #noise
            noiseXerr,syndromeXerr=mXerr.add_noise()
            noiseZerr,syndromeZerr=mZerr.add_noise()

            #correction
            correctionXerr=mXerr.decode(syndromeXerr)
            correctionZerr=mZerr.decode(syndromeZerr)

            #correction + noise 
            corr_noise_Xerr=np.array((correctionXerr+noiseXerr)%2)
            corr_noise_Zerr=np.array((correctionZerr+noiseZerr)%2)

            #correction + noise action on the logicals
            if periodic==1:
                sum_logicalZ1=corr_noise_Xerr[Z_logical1_Graph_Xerr]
                sum_logicalX1=corr_noise_Zerr[X_logical1_Graph_Zerr]
                sum_logicalZ2=corr_noise_Xerr[Z_logical2_Graph_Xerr]
                sum_logicalX2=corr_noise_Zerr[X_logical2_Graph_Zerr]
                failure_list[i]=sum(sum_logicalZ1)%2 or sum(sum_logicalZ2)%2 or sum(sum_logicalX1)%2 or sum(sum_logicalX2)%2

            else:
                sum_logicalZ1=corr_noise_Xerr[Z_logical1_Graph_Xerr]
                sum_logicalX1=corr_noise_Zerr[X_logical1_Graph_Zerr]
                failure_list[i]=sum(sum_logicalZ1)%2 or sum(sum_logicalX1)%2

        pL_rand[realization_index]=failure_list.mean()
        std_rand[realization_index]=failure_list.std()/np.sqrt(num_mc)

    pL=np.sum(pL_rand)/num_realiz
    std=np.sqrt(np.sum(vsquare(std_rand)))/num_realiz
    print(pL,std)
    return [pL,std]

def pymatching_rand(bias,bias_str,num_mc,L_list,error_probabilities,pH,pitch,num_realiz,periodic):

    pL_list=np.zeros((len(L_list),len(error_probabilities)))
    std_list=np.zeros((len(L_list),len(error_probabilities)))        
    log_pL_list=np.zeros((len(L_list),len(error_probabilities)))
    log_std_list=np.zeros((len(L_list),len(error_probabilities)))
    error_model = BiasedDepolarizingErrorModel(bias,bias_str)

    for ind_L in range(0,len(L_list)):
        L=L_list[ind_L]
        print('L=',L)
        Nx=L
        Ny=L

        prob_dist=0.2,0.2,0.2,0.4
        Graph_Zerr=graph_temp(Nx,Ny,prob_dist,1)
        if pH==1.5:
            hadamard_vec=np.zeros((len(Graph_Zerr.edges)))
            for (u,v) in list(Graph_Zerr.edges):
                if abs(u-v)==1 or abs(u-v)==Nx-1:
                    hadamard_vec[Graph_Zerr[u][v]['qubit_id']]=1

        if pH==2.5:       
            hadamard_vec=np.zeros((len(Graph_Zerr.edges)))
            for (u,v) in list(Graph_Zerr.edges):
                if abs(u-v)==1 or abs(u-v)==Nx-1:
                    hadamard_vec[Graph_Zerr[u][v]['qubit_id']]=1
            
            spacefillcurve_extra_hadamard=spiral(Nx,pitch)

            for (u,v) in spacefillcurve_extra_hadamard:
                    hadamard_vec[Graph_Zerr[u][v]['qubit_id']]=(hadamard_vec[Graph_Zerr[u][v]['qubit_id']]+1)%2 

        p=mp.Pool()
        func=partial(parallel_step_p,bias,bias_str,error_model,Nx,Ny,num_mc,pH,hadamard_vec,num_realiz,periodic)
        result=p.map(func,error_probabilities)
        p.close()
        p.join()

        for i in range(len(result)):
            pL_list[ind_L][i]=result[i][0]
            std_list[ind_L][i]=result[i][1]
            log_pL_list[ind_L][i]=-np.log(pL_list[ind_L][i])
            log_std_list[ind_L][i]=std_list[ind_L][i]/(pL_list[ind_L][i]*np.log(10))

    return [pL_list,std_list,log_pL_list,log_std_list]







def parallel_step_pX(bias,Nx,Ny,num_mc,pH,hadamard_vec,pX): 

    failure_list=np.zeros(num_mc) 

    error_model = BiasedDepolarizingErrorModel(bias,'Z')
    prob_dist= error_model.probability_distribution(pX)
    pI,pX,pY,pZ=prob_dist

    periodic=1
    [Graph_Xerr,Graph_Zerr,X_logical1_Graph_Zerr,X_logical2_Graph_Zerr,Z_logical1_Graph_Xerr,Z_logical2_Graph_Xerr]=construct_lattice(Nx,Ny,prob_dist,periodic)

    if pH!=0:
        [Graph_Xerr,Graph_Zerr]=weight_error_changes(prob_dist,Graph_Xerr,Graph_Zerr,periodic,hadamard_vec)

    mXerr = Matching(Graph_Xerr)
    mZerr = Matching(Graph_Zerr)

    for i in range(num_mc):
        #noise
        noiseXerr,syndromeXerr = mXerr.add_noise()
        noiseZerr,syndromeZerr = mZerr.add_noise()

        #correction
        correctionXerr = mXerr.decode(syndromeXerr)
        correctionZerr = mZerr.decode(syndromeZerr)

        #correction + noise 
        corr_noise_Xerr = np.array((correctionXerr+noiseXerr)%2)
        corr_noise_Zerr = np.array((correctionZerr+noiseZerr)%2)

        #correction + noise action on the logicals
        sum_logicalZ1 = corr_noise_Xerr[Z_logical1_Graph_Xerr]
        sum_logicalZ2 = corr_noise_Xerr[Z_logical2_Graph_Xerr]
        sum_logicalX1 = corr_noise_Zerr[X_logical1_Graph_Zerr]
        sum_logicalX2 = corr_noise_Zerr[X_logical2_Graph_Zerr]
  
        failure_list[i]= sum(sum_logicalZ1)%2 or sum(sum_logicalZ2)%2 or sum(sum_logicalX1)%2 or sum(sum_logicalX2)%2

    pL=failure_list.mean()
    std=failure_list.std()/np.sqrt(num_mc)

    return [pL,std]

def pymatching_rand2(bias,num_mc,L_list,error_probabilities,pH,pitch,num_codes):

    pL_list_avg=np.zeros((len(L_list),len(error_probabilities)))
    std_list_avg=np.zeros((len(L_list),len(error_probabilities)))
   
    log_pL_list=np.zeros((len(L_list),len(error_probabilities)))
    log_std_list=np.zeros((len(L_list),len(error_probabilities)))

    pX_list=error_probabilities

    for ind_L in range(0,len(L_list)):
        L=L_list[ind_L]
        print('L=',L)
        Nx=L
        Ny=L

        prob_dist=0.2,0.2,0.2,0.4
        Graph_Zerr=graph_temp(Nx,Ny,prob_dist,1)
        if pH==1.5:
            hadamard_vec=np.zeros((len(Graph_Zerr.edges)))
            for (u,v) in list(Graph_Zerr.edges):
                if abs(u-v)==1 or abs(u-v)==Nx-1:
                    hadamard_vec[Graph_Zerr[u][v]['qubit_id']]=1

        if pH==2.5:       
            hadamard_vec=np.zeros((len(Graph_Zerr.edges)))
            for (u,v) in list(Graph_Zerr.edges):
                if abs(u-v)==1 or abs(u-v)==Nx-1:
                    hadamard_vec[Graph_Zerr[u][v]['qubit_id']]=1
            
            spacefillcurve_extra_hadamard=spiral(Nx,pitch)

            for (u,v) in spacefillcurve_extra_hadamard:
                    hadamard_vec[Graph_Zerr[u][v]['qubit_id']]=(hadamard_vec[Graph_Zerr[u][v]['qubit_id']]+1)%2 

        pL_list =np.zeros(len(pX_list))
        std_list =np.zeros(len(pX_list))    
        
        p=mp.Pool()
        func=partial(parallel_step_pX,bias,Nx,Ny,num_mc,pH,hadamard_vec)
        result=p.map(func,pX_list)
        p.close()
        p.join()
        print(result)

        for i in range(len(result)):
            pL_list[i]=result[i][0]
            std_list[i]=result[i][1]
            log_pL_list[ind_L][i]=-np.log(pL_list[i])
            log_std_list[ind_L][i]=std_list[i]/(pL_list[i]*np.log(10))

        [pL_list_avg[ind_L],std_list_avg[ind_L]]=[pL_list,std_list]
    
    print(pL_list_avg)
    print(std_list_avg)

    return [pL_list_avg,std_list_avg,log_pL_list,log_std_list]

In [567]:
p_min,p_max,dp=0.10,0.50,0.015
error_probabilities=np.linspace(p_min,p_max,18);
Nx=3
Ny=3
pH=1.5
periodic=1
    
for error_probability in error_probabilities:
    error_model = BiasedDepolarizingErrorModel(1000,'X')
    prob_dist= error_model.probability_distribution(error_probability)


    hadamard_vec=func_hadamard_vec(Nx,Ny,pH,prob_dist,periodic)
    Graph_Zerr=graph_temp(Nx,Ny,prob_dist,1)
    if pH==1.5:
        hadamard_vec2=np.zeros((len(Graph_Zerr.edges)))
        for (u,v) in list(Graph_Zerr.edges):
            if abs(u-v)==1 or abs(u-v)==Nx-1:
                hadamard_vec2[Graph_Zerr[u][v]['qubit_id']]=1
    print(hadamard_vec==hadamard_vec2)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True]
[

In [568]:
error_model = BiasedDepolarizingErrorModel(1000,'X')
periodic=0
prob_dist= error_model.probability_distribution(0.2)
[Graph_Xerr,Graph_Zerr,X_logical1_Graph_Zerr,Z_logical1_Graph_Xerr]=construct_lattice(Nx,Ny,prob_dist,periodic)
print(X_logical1_Graph_Zerr)
print(Z_logical1_Graph_Xerr)

[2 5 7]
[ 2  0  8 13]


In [569]:
for u,v in Graph_Xerr.edges:
    print(u,v,Graph_Xerr[u][v]['qubit_id'])

8 1 0
8 0 2
8 2 8
8 3 13
8 9 -1
1 0 1
1 5 3
1 2 9
0 4 5
5 4 4
5 9 6
5 6 11
4 9 7
9 6 12
9 7 17
2 6 10
2 3 14
6 7 16
3 7 15


In [570]:

for u,v in Graph_Zerr.edges:
    print(u,v,Graph_Zerr[u][v]['qubit_id'])

0 1 0
0 3 1
0 9 2
1 2 8
1 4 9
3 4 3
3 6 4
3 9 5
9 6 7
9 10 -1
4 5 10
4 7 11
6 7 6
7 8 12
2 10 13
2 5 14
5 10 15
5 8 16
8 10 17


In [573]:
(-1)*np.ones((3,3))

array([[-1., -1., -1.],
       [-1., -1., -1.],
       [-1., -1., -1.]])

In [774]:

def rotdic_pbc(Nx,Ny):

    v=np.zeros((Nx,Ny),dtype=int)

    for i in range(0,Nx):
        for j in range(0,Ny):
            if (i+j)%2==0: 
                if j%2==0:
                    v[i,j]=i/2+((j-1)%Nx)*Nx/2
                else:
                    v[i,j]=((i-1)%Nx)/2+((j-1)%Nx)*Nx/2
            if (i+j)%2==1:
                if j%2==0:
                    v[i,j]=((i-1)%Nx)/2+((j-1)%Nx)*Nx/2
                else:
                    v[i,j]=i/2+((j-1)%Nx)*Nx/2     
    return v


def rotdic_obc(Nx,Ny):

    Zstabs_num=(Nx/2+1)*Ny
    Xstabs_num=Nx/2*(Ny+2)
    
    virZ1,virZ2,virZ3,virZ4=np.arange(Zstabs_num,Zstabs_num+4,dtype=int)
    virX1,virX2,virX3,virX4=np.arange(Xstabs_num,Xstabs_num+4,dtype=int)
        
    vX=(-10)*np.ones((Nx+4,Ny+4),dtype=int)

    for i in range(1,Nx+3):
        for j in range(1,Ny+3):
            if (i+j)%2==0: 
                if j==1:
                    if i%4==1:
                        vX[i,j]=virX1
                    else:
                        vX[i,j]=virX2
                elif j==Ny+2:
                    if i%4==0:                        
                        vX[i,j]=virX3
                    else:
                        vX[i,j]=virX4                        
                else:
                    if j%2==0:
                        vX[i,j]=(i-2)/2+(j-2)*(Nx/2+1)
                    else:
                        vX[i,j]=(i-1)/2+(j-2)*(Nx/2+1)


    vZ=(-10)*np.ones((Nx+4,Ny+4),dtype=int)

    for i in range(1,Nx+3):
        for j in range(1,Ny+3):
            if (i+j)%2==1:
                if i==1:
                    if j%4==0:
                        vZ[i,j]=virZ1
                    else:
                        vZ[i,j]=virZ2
                elif i==Nx+2:
                    if j%4==1:
                        vZ[i,j]=virZ3
                    else:
                        vZ[i,j]=virZ4
                else:
                    vZ[i,j]=(i-2)/2+(j-1)*Nx/2

    return [vX,vZ]

def construct_rot_lattice(Nx,Ny,prob_dist,periodic):

    pI,pX,pY,pZ=prob_dist
    #weights
    wZ=abs(np.log(pZ/pI))
    wX=abs(np.log(pX/pI))

    if periodic==0:
        #even times even stabilizer number lattice
        Xstabs_num=(Nx/2+1)*Ny
        Zstabs_num=Nx/2*(Ny+2)

        virZ1,virZ2,virZ3,virZ4=np.arange(Zstabs_num,Zstabs_num+4,dtype=int)
        virX1,virX2,virX3,virX4=np.arange(Xstabs_num,Xstabs_num+4,dtype=int)
        [vX,vZ]=rotdic_obc(Nx,Ny)

        Graph_Zerr=nx.MultiGraph()
        Graph_Xerr=nx.MultiGraph()

        X_logical1_Graph_Zerr=np.array([],dtype=int)
        Z_logical1_Graph_Xerr=np.array([],dtype=int)

        id=0
        for i in range(1,Nx+3):
            for j in range(1,Ny+3): 
                if (i+j)%2==1:
                    if vZ[i+1,j+1]>-1:
#                         print(i,j,vZ[i,j],vZ[i+1,j+1])
                        Graph_Zerr.add_edge(vZ[i,j],vZ[i+1,j+1],qubit_id=id,weight=wZ,error_probability=pZ)
                        Graph_Xerr.add_edge(vX[i,j+1],vX[i+1,j],qubit_id=id,weight=wX,error_probability=pX)
                        id+=1
                    if vZ[i+1,j-1]>-1:
#                         print(i,j,vZ[i,j],vZ[i+1,j-1])                        
                        Graph_Zerr.add_edge(vZ[i,j],vZ[i+1,j-1],qubit_id=id,weight=wZ,error_probability=pZ)
                        Graph_Xerr.add_edge(vX[i,j-1],vX[i+1,j],qubit_id=id,weight=wX,error_probability=pX)
                        id+=1
                    
        Graph_Zerr.add_edge(virZ1,virZ2,qubit_id=-1,weight=0,error_probability=0)  
        Graph_Zerr.add_edge(virZ3,virZ4,qubit_id=-2,weight=0,error_probability=0)  
        Graph_Zerr.add_edge(virZ1,virZ4,qubit_id=-3,weight=0,error_probability=0)  
        Graph_Xerr.add_edge(virX1,virX2,qubit_id=-1,weight=0,error_probability=0) 
        Graph_Xerr.add_edge(virX3,virX4,qubit_id=-2,weight=0,error_probability=0) 
        Graph_Xerr.add_edge(virX1,virX4,qubit_id=-3,weight=0,error_probability=0) 

        Z_logical_qubit_num=Nx+1  
        X_logical_qubit_num=Ny    

        for j in range(1,Ny+3):
            if (2+j)%2==1:
                if vZ[3,j+1]>-1:
#                     print(i,j,vZ[2,j],vZ[1,j+1],Graph_Zerr[vZ[2,j]][vZ[1,j+1],'qubit_id'])
                    X_logical1_Graph_Zerr=np.append(X_logical1_Graph_Zerr,Graph_Zerr[vZ[2,j]][vZ[3,j+1]]['qubit_id'])
                if vZ[3,j-1]>-1:              
                    X_logical1_Graph_Zerr=np.append(X_logical1_Graph_Zerr,Graph_Zerr[vZ[2,j]][vZ[3,j-1]]['qubit_id'])

        for i in range(1,Nx+3):
            if (i+2)%2==0:
                if vX[i+1,3]>-1:
                    Z_logical1_Graph_Xerr=np.append(Z_logical1_Graph_Xerr,Graph_Xerr[vX[i,2]][vX[i+1,3]]['qubit_id'])
                if vX[i-1,3]>-1:
                    Z_logical1_Graph_Xerr=np.append(Z_logical1_Graph_Xerr,Graph_Xerr[vX[i,2]][vX[i-1,3]]['qubit_id'])

        Graph_Zerr.nodes[virZ1]['is_boundary']=True
        Graph_Zerr.nodes[virZ2]['is_boundary']=True
        Graph_Zerr.nodes[virZ3]['is_boundary']=True
        Graph_Zerr.nodes[virZ4]['is_boundary']=True
        Graph_Xerr.nodes[virX1]['is_boundary']=True   
        Graph_Xerr.nodes[virX2]['is_boundary']=True   
        Graph_Xerr.nodes[virX3]['is_boundary']=True   
        Graph_Xerr.nodes[virX4]['is_boundary']=True   

        return [Graph_Xerr,Graph_Zerr,X_logical1_Graph_Zerr,Z_logical1_Graph_Xerr]

    else: 
        v=rotdic_pbc(Nx,Ny)

        Graph_Zerr=nx.Graph()
        Graph_Xerr=nx.Graph()
        
        #Nx and Ny are number of stabilizers

        if (Nx%2==0 and Ny%2==0):
            X_logical1_Graph_Zerr=np.array([],dtype=int)
            X_logical2_Graph_Zerr=np.array([],dtype=int)
            Z_logical1_Graph_Xerr=np.array([],dtype=int)
            Z_logical2_Graph_Xerr=np.array([],dtype=int)

            id=0
            for i in range(0,Nx):
                for j in range(0,Ny):
                    if (i+j)%2==0:
                        Graph_Zerr.add_edge(v[i,j],v[(i+1)% Nx,(j+1)%Ny],qubit_id=id,weight=wZ,error_probability=pZ)
                        Graph_Xerr.add_edge(v[i,(j+1)%Ny],v[(i+1)%Nx,j],qubit_id=id,weight=wX,error_probability=pX)
                        id+=1
                        Graph_Zerr.add_edge(v[i,j],v[(i+1)%Nx,(j-1)%Ny],qubit_id=id,weight=wZ,error_probability=pZ)
                        Graph_Xerr.add_edge(v[i,(j-1)%Ny],v[(i+1)%Nx,j],qubit_id=id,weight=wX,error_probability=pX)
                        id+=1

            for i in range(0,Nx):
                for j in range(0,Ny):
                    if (i+j)%2==0 and i==0:
                        X_logical1_Graph_Zerr=np.append(X_logical1_Graph_Zerr,Graph_Zerr[v[i,j]][v[(i+1)%Nx,(j+1)%Ny]]['qubit_id'])                        
                        X_logical1_Graph_Zerr=np.append(X_logical1_Graph_Zerr,Graph_Zerr[v[i,j]][v[(i+1)%Nx,(j-1)%Ny]]['qubit_id'])                        
                    if (i+j)%2==0 and j==0:
                        X_logical2_Graph_Zerr=np.append(X_logical2_Graph_Zerr,Graph_Zerr[v[i,j]][v[(i+1)%Nx,(j+1)%Ny]]['qubit_id'])
                        X_logical2_Graph_Zerr=np.append(X_logical2_Graph_Zerr,Graph_Zerr[v[i,j]][v[(i-1)%Nx,(j+1)%Ny]]['qubit_id'])
                    if (i+j)%2==1 and i==0:
                        Z_logical2_Graph_Xerr=np.append(Z_logical2_Graph_Xerr,Graph_Xerr[v[i,j]][v[(i+1)%Nx,(j+1)%Ny]]['qubit_id'])
                        Z_logical2_Graph_Xerr=np.append(Z_logical2_Graph_Xerr,Graph_Xerr[v[i,j]][v[(i+1)%Nx,(j-1)%Ny]]['qubit_id'])
                    if (i+j)%2==1 and j==0:
                        Z_logical1_Graph_Xerr=np.append(Z_logical1_Graph_Xerr,Graph_Xerr[v[i,j]][v[(i+1)%Nx,(j+1)%Ny]]['qubit_id'])
                        Z_logical1_Graph_Xerr=np.append(Z_logical1_Graph_Xerr,Graph_Xerr[v[i,j]][v[(i-1)%Nx,(j+1)%Ny]]['qubit_id'])

            return [Graph_Xerr,Graph_Zerr,X_logical1_Graph_Zerr,X_logical2_Graph_Zerr,Z_logical1_Graph_Xerr,Z_logical2_Graph_Xerr]
    
        elif(Nx%2!=Ny%2):
            print('the modified decoder graphs and logicals (which are now a Y-Z or Y-X pair) needs to be implemented below')
            X_logical1_Graph_Zerr=np.array([],dtype=int)
            Z_logical1_Graph_Xerr=np.array([],dtype=int)

            id=0
            for i in range(0,Nx):
                for j in range(0,Ny):
                    if (i+j)%2==0:
                        Graph_Zerr.add_edge(v[i,j],v[(i+1)% Nx,(j+1)%Ny],qubit_id=id,weight=wZ,error_probability=pZ)
                        Graph_Xerr.add_edge(v[i,(j+1)%Ny],v[(i+1)%Nx,j],qubit_id=id,weight=wX,error_probability=pX)
                        id+=1
                        Graph_Zerr.add_edge(v[i,j],v[(i+1)%Nx,(j-1)%Ny],qubit_id=id,weight=wZ,error_probability=pZ)
                        Graph_Xerr.add_edge(v[i,(j-1)%Ny],v[(i+1)%Nx,j],qubit_id=id,weight=wX,error_probability=pX)
                        id+=1
            
            
            for i in range(0,Nx):
                for j in range(0,Ny):
                    if v[i,j] in range(0,Nx*Ny,Nx):
                        X_logical1_Graph_Zerr=np.append(X_logical1_Graph_Zerr,Graph_Zerr[v[i,j]][v[(i+1)%Nx,(j+1)%Ny],'qubit_id'])
                    if v[i,j] in range(0,Nx):
                        Z_logical1_Graph_Xerr=np.append(Z_logical1_Graph_Xerr,Graph_Xerr[v[i,j]][v[(i+1)%Nx,(j+1)%Ny],'qubit_id'])

            return [Graph_Xerr,Graph_Zerr,X_logical1_Graph_Zerr,Z_logical1_Graph_Xerr]



In [771]:
prob_dist=[0.1,0.3,0.3,0.3]
[g1,g2,X_logical1_Graph_Zerr,Z_logical1_Graph_Xerr]=construct_rot_lattice(4,4,prob_dist,0)
for u,v in g1.edges:
    print(u,v,g1[u][v]['qubit_id'])
print(X_logical1_Graph_Zerr)
for u,v in g2.edges:
    print(u,v,g2[u][v]['qubit_id'])

print(len(g1.edges),len(g2.edges))
print(Z_logical1_Graph_Xerr)
g1.nodes

3 0 0
3 6 3
0 12 1
0 13 5
0 4 7
12 1 15
12 2 21
12 13 -1
12 15 -3
9 6 2
9 15 4
6 4 6
6 10 9
15 10 14
15 11 24
15 14 -2
13 1 11
4 1 10
4 7 13
10 7 12
1 5 17
7 5 16
7 11 19
5 2 20
5 8 23
11 8 22
[5 6 7 8 9]
13 4 0
13 0 1
13 8 4
13 12 -1
4 12 3
4 6 6
4 2 7
0 2 5
12 8 2
12 15 -3
8 10 8
8 6 9
2 5 10
2 1 11
6 9 12
6 5 13
10 9 14
5 7 16
5 3 17
1 3 15
9 11 18
9 7 19
3 15 21
7 15 23
11 15 24
15 14 -2
26 26
[ 7  0 17 10 20]


NodeView((3, 0, 12, 9, 6, 15, 13, 4, 10, 1, 7, 5, 11, 2, 8, 14))

In [772]:
i=0
j=2
Nx=4
Ny=4
v=rotdic_pbc(Nx,Ny)
Xstabs_num=(Nx/2+1)*Ny
Zstabs_num=Nx/2*(Ny+2)
print(Xstabs_num,Zstabs_num)
print(v.T)

12.0 12.0
[[6 6 7 7]
 [0 0 1 1]
 [2 2 3 3]
 [4 4 5 5]]


In [773]:
[vx,vz]=rotdic_obc(4,4)
vz.T

array([[-10, -10, -10, -10, -10, -10, -10, -10],
       [-10, -10,   0, -10,   1, -10,  15, -10],
       [-10,  13, -10,   2, -10,   3, -10, -10],
       [-10, -10,   4, -10,   5, -10,  15, -10],
       [-10,  12, -10,   6, -10,   7, -10, -10],
       [-10, -10,   8, -10,   9, -10,  15, -10],
       [-10,  13, -10,  10, -10,  11, -10, -10],
       [-10, -10, -10, -10, -10, -10, -10, -10]])

In [648]:
g2[1][2]

{'qubit_id': 11, 'weight': 1.0986122886681096, 'error_probability': 0.3}

In [608]:
def rotdic_obc(Nx,Ny):

    Zstabs_num=(Nx/2+1)*Ny
    Xstabs_num=Nx/2*(Ny+2)

    virZ1,virZ2=Zstabs_num,Zstabs_num+1
    virX1,virX2=Xstabs_num,Xstabs_num+1
        
    vX=(-1)*np.ones((Nx+4,Ny+4),dtype=int)

    for i in range(1,Nx+3):
        for j in range(1,Ny+3):
            if (i+j)%2==0: 
                if j==1:
                    vX[i,j]=virX1
                elif j==Ny+2:
                    vX[i,j]=virX2
                else:
                    if j%2==0:
                        vX[i,j]=(i-2)/2+(j-2)*(Nx/2+1)
                    else:
                        vX[i,j]=(i-1)/2+(j-2)*(Nx/2+1)


    vZ=(-1)*np.ones((Nx+4,Ny+4),dtype=int)

    for i in range(1,Nx+3):
        for j in range(0,Ny+3):
            if (i+j)%2==1:
                if i==1:
                    vZ[i,j]=virZ1
                elif i==Nx+2:
                    vZ[i,j]=virZ2
                else:
                    vZ[i,j]=(i-2)/2+(j-1)*Nx/2

    return [vX,vZ]

[vX,vZ]=rotdic_obc(6,6)
print(vX.T)
print(vZ.T)


[[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 24 -1 24 -1 24 -1 24 -1 -1]
 [-1 -1  0 -1  1 -1  2 -1  3 -1]
 [-1  4 -1  5 -1  6 -1  7 -1 -1]
 [-1 -1  8 -1  9 -1 10 -1 11 -1]
 [-1 12 -1 13 -1 14 -1 15 -1 -1]
 [-1 -1 16 -1 17 -1 18 -1 19 -1]
 [-1 20 -1 21 -1 22 -1 23 -1 -1]
 [-1 -1 25 -1 25 -1 25 -1 25 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]]
[[-1 24 -1 -2 -1 -1 -1  0 -1 -1]
 [-1 -1  0 -1  1 -1  2 -1 25 -1]
 [-1 24 -1  3 -1  4 -1  5 -1 -1]
 [-1 -1  6 -1  7 -1  8 -1 25 -1]
 [-1 24 -1  9 -1 10 -1 11 -1 -1]
 [-1 -1 12 -1 13 -1 14 -1 25 -1]
 [-1 24 -1 15 -1 16 -1 17 -1 -1]
 [-1 -1 18 -1 19 -1 20 -1 25 -1]
 [-1 24 -1 21 -1 22 -1 23 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]]


In [775]:
Nx,Ny=4,4

pI,pX,pY,pZ=prob_dist
#weights
wZ=abs(np.log(pZ/pI))
wX=abs(np.log(pX/pI))
    
Xstabs_num=(Nx/2+1)*Ny
Zstabs_num=Nx/2*(Ny+2)

virZ1,virZ2,virZ3,virZ4=np.arange(Zstabs_num,Zstabs_num+4,dtype=int)
virX1,virX2,virX3,virX4=np.arange(Xstabs_num,Xstabs_num+4,dtype=int)
[vX,vZ]=rotdic_obc(Nx,Ny)

Graph_Zerr=nx.Graph()
Graph_Xerr=nx.Graph()

X_logical1_Graph_Zerr=np.array([],dtype=int)
Z_logical1_Graph_Xerr=np.array([],dtype=int)

id=0
for i in range(1,Nx+3):
    for j in range(1,Ny+3): 
        if (i+j)%2==1:
            if vZ[i+1,j+1]>-1:
#                         print(i,j,vZ[i,j],vZ[i+1,j+1])
                Graph_Zerr.add_edge(vZ[i,j],vZ[i+1,j+1],qubit_id=id,weight=wZ,error_probability=pZ)
                Graph_Xerr.add_edge(vX[i,j+1],vX[i+1,j],qubit_id=id,weight=wX,error_probability=pX)
                id+=1
            if vZ[i+1,j-1]>-1:
#                         print(i,j,vZ[i,j],vZ[i+1,j-1])                        
                Graph_Zerr.add_edge(vZ[i,j],vZ[i+1,j-1],qubit_id=id,weight=wZ,error_probability=pZ)
                Graph_Xerr.add_edge(vX[i,j-1],vX[i+1,j],qubit_id=id,weight=wX,error_probability=pX)
                id+=1

Graph_Zerr.add_edge(virZ1,virZ2,qubit_id=-1,weight=0,error_probability=0)  
Graph_Zerr.add_edge(virZ3,virZ4,qubit_id=-2,weight=0,error_probability=0)  
Graph_Zerr.add_edge(virZ1,virZ4,qubit_id=-3,weight=0,error_probability=0)  
Graph_Xerr.add_edge(virX1,virX2,qubit_id=-1,weight=0,error_probability=0) 
Graph_Xerr.add_edge(virX3,virX4,qubit_id=-2,weight=0,error_probability=0) 
Graph_Xerr.add_edge(virX1,virX4,qubit_id=-3,weight=0,error_probability=0) 

Z_logical_qubit_num=Nx+1  
X_logical_qubit_num=Ny    

for j in range(1,Ny+3):
    if (2+j)%2==1:
        if vZ[3,j+1]>-1:
#                     print(i,j,vZ[2,j],vZ[1,j+1],Graph_Zerr[vZ[2,j]][vZ[1,j+1],'qubit_id'])
            X_logical1_Graph_Zerr=np.append(X_logical1_Graph_Zerr,Graph_Zerr[vZ[2,j]][vZ[3,j+1]]['qubit_id'])
        if vZ[3,j-1]>-1:              
            X_logical1_Graph_Zerr=np.append(X_logical1_Graph_Zerr,Graph_Zerr[vZ[2,j]][vZ[3,j-1]]['qubit_id'])

for i in range(1,Nx+3):
    if (i+2)%2==0:
        if vX[i+1,3]>-1:
            Z_logical1_Graph_Xerr=np.append(Z_logical1_Graph_Xerr,Graph_Xerr[vX[i,2]][vX[i+1,3]]['qubit_id'])
        if vX[i-1,3]>-1:
            Z_logical1_Graph_Xerr=np.append(Z_logical1_Graph_Xerr,Graph_Xerr[vX[i,2]][vX[i-1,3]]['qubit_id'])

Graph_Zerr.nodes[virZ1]['is_boundary']=True
Graph_Zerr.nodes[virZ2]['is_boundary']=True
Graph_Zerr.nodes[virZ3]['is_boundary']=True
Graph_Zerr.nodes[virZ4]['is_boundary']=True
Graph_Xerr.nodes[virX1]['is_boundary']=True   
Graph_Xerr.nodes[virX2]['is_boundary']=True   
Graph_Xerr.nodes[virX3]['is_boundary']=True   
Graph_Xerr.nodes[virX4]['is_boundary']=True   

In [776]:
for u,v in Graph_Xerr.edges:
    print(u,v,Graph_Xerr[u][v]['qubit_id'])
print(X_logical1_Graph_Zerr)
for u,v in Graph_Zerr.edges:
    print(u,v,Graph_Zerr[u][v]['qubit_id'])

print(len(Graph_Xerr.edges),len(Graph_Zerr.edges))
print(Z_logical1_Graph_Xerr)
g1.nodes

3 0 0
3 6 3
0 12 1
0 13 5
0 4 7
12 1 15
12 2 21
12 13 -1
12 15 -3
9 6 2
9 15 4
6 4 6
6 10 9
15 10 8
15 11 24
15 14 -2
13 1 11
4 1 10
4 7 13
10 7 12
10 14 14
1 5 17
7 5 16
7 11 19
14 11 18
5 2 20
5 8 23
11 8 22
[5 6 7 8 9]
13 4 0
13 0 1
13 8 4
13 12 -1
4 12 3
4 6 6
4 2 7
0 2 5
12 8 2
12 15 -3
8 10 8
8 6 9
2 5 10
2 1 11
6 9 12
6 5 13
10 9 14
5 7 16
5 3 17
1 3 15
9 11 18
9 7 19
3 15 20
3 14 21
7 14 22
7 15 23
11 14 24
15 14 -2
28 28
[ 7  0 17 10 20]


NodeView((3, 0, 12, 9, 6, 15, 13, 4, 10, 1, 7, 5, 11, 2, 8, 14))

In [797]:

def rotdic_pbc(Nx,Ny):

    v=np.zeros((Nx,Ny),dtype=int)

    for i in range(0,Nx):
        for j in range(0,Ny):
            if (i+j)%2==0: 
                if j%2==0:
                    v[i,j]=i/2+((j-1)%Nx)*Nx/2
                else:
                    v[i,j]=((i-1)%Nx)/2+((j-1)%Nx)*Nx/2
            if (i+j)%2==1:
                if j%2==0:
                    v[i,j]=((i-1)%Nx)/2+((j-1)%Nx)*Nx/2
                else:
                    v[i,j]=i/2+((j-1)%Nx)*Nx/2     
    return v


def rotdic_obc(Nx,Ny):

    Zstabs_num=(Nx/2+1)*Ny
    Xstabs_num=Nx/2*(Ny+2)   
    
    virZ1,virZ2=Zstabs_num,Zstabs_num+1
    virX1,virX2=Xstabs_num,Xstabs_num+1
    vX=(-10)*np.ones((Nx+4,Ny+4),dtype=int)

    for i in range(1,Nx+3):
        for j in range(1,Ny+3):
            if (i+j)%2==0: 
                if j==1:
                    vX[i,j]=virX1
                elif j==Ny+2:
                    vX[i,j]=virX2                      
                else:
                    if j%2==0:
                        vX[i,j]=(i-2)/2+(j-2)*(Nx/2+1)
                    else:
                        vX[i,j]=(i-1)/2+(j-2)*(Nx/2+1)


    vZ=(-10)*np.ones((Nx+4,Ny+4),dtype=int)

    for i in range(1,Nx+3):
        for j in range(1,Ny+3):
            if (i+j)%2==1:
                if i==1:
                    vZ[i,j]=virZ1
                elif i==Nx+2:
                    vZ[i,j]=virZ2      
                else:
                    vZ[i,j]=(i-2)/2+(j-1)*Nx/2

    return [vX,vZ]

def construct_rot_lattice(Nx,Ny,prob_dist,periodic):

    pI,pX,pY,pZ=prob_dist
    #weights
    wZ=abs(np.log(pZ/pI))
    wX=abs(np.log(pX/pI))

    if periodic==0:
        #even times even stabilizer number lattice
        Xstabs_num=(Nx/2+1)*Ny
        Zstabs_num=Nx/2*(Ny+2)

        virZ1,virZ2=Zstabs_num,Zstabs_num+1
        virX1,virX2=Xstabs_num,Xstabs_num+1
        
        [vX,vZ]=rotdic_obc(Nx,Ny)

        Graph_Zerr=nx.MultiGraph()
        Graph_Xerr=nx.MultiGraph()

        X_logical1_Graph_Zerr=np.array([],dtype=int)
        Z_logical1_Graph_Xerr=np.array([],dtype=int)

        id=0
        for i in range(1,Nx+3):
            for j in range(1,Ny+3): 
                if (i+j)%2==1:
                    if vZ[i+1,j+1]>-1:
#                         print(i,j,vZ[i,j],vZ[i+1,j+1])
                        Graph_Zerr.add_edge(vZ[i,j],vZ[i+1,j+1],qubit_id=id,weight=wZ,error_probability=pZ)
                        Graph_Xerr.add_edge(vX[i,j+1],vX[i+1,j],qubit_id=id,weight=wX,error_probability=pX)
                        id+=1
                    if vZ[i+1,j-1]>-1:
#                         print(i,j,vZ[i,j],vZ[i+1,j-1])                        
                        Graph_Zerr.add_edge(vZ[i,j],vZ[i+1,j-1],qubit_id=id,weight=wZ,error_probability=pZ)
                        Graph_Xerr.add_edge(vX[i,j-1],vX[i+1,j],qubit_id=id,weight=wX,error_probability=pX)
                        id+=1
        
        print('id=',id)
        Graph_Zerr.add_edge(virZ1,virZ2,qubit_id=-1,weight=0,error_probability=0)  
        Graph_Xerr.add_edge(virX1,virX2,qubit_id=-1,weight=0,error_probability=0) 

        Z_logical_qubit_num=Nx+1  
        X_logical_qubit_num=Ny    

        for j in range(1,Ny+3):
            if (2+j)%2==1:
                if vZ[3,j+1]>-1:
                    X_logical1_Graph_Zerr=np.append(X_logical1_Graph_Zerr,Graph_Zerr[vZ[2,j]][vZ[3,j+1]][0]['qubit_id'])
                if vZ[3,j-1]>-1:              
                    X_logical1_Graph_Zerr=np.append(X_logical1_Graph_Zerr,Graph_Zerr[vZ[2,j]][vZ[3,j-1]][0]['qubit_id'])

        for i in range(1,Nx+3):
            if (i+2)%2==0:
                if vX[i+1,3]>-1:
                    Z_logical1_Graph_Xerr=np.append(Z_logical1_Graph_Xerr,Graph_Xerr[vX[i,2]][vX[i+1,3]][0]['qubit_id'])
                if vX[i-1,3]>-1:
                    Z_logical1_Graph_Xerr=np.append(Z_logical1_Graph_Xerr,Graph_Xerr[vX[i,2]][vX[i-1,3]][0]['qubit_id'])

        Graph_Zerr.nodes[virZ1]['is_boundary']=True
        Graph_Zerr.nodes[virZ2]['is_boundary']=True
        Graph_Xerr.nodes[virX1]['is_boundary']=True   
        Graph_Xerr.nodes[virX2]['is_boundary']=True    

        return [Graph_Xerr,Graph_Zerr,X_logical1_Graph_Zerr,Z_logical1_Graph_Xerr]

    else: 
        v=rotdic_pbc(Nx,Ny)

        Graph_Zerr=nx.Graph()
        Graph_Xerr=nx.Graph()
        
        #Nx and Ny are number of stabilizers

        if (Nx%2==0 and Ny%2==0):
            X_logical1_Graph_Zerr=np.array([],dtype=int)
            X_logical2_Graph_Zerr=np.array([],dtype=int)
            Z_logical1_Graph_Xerr=np.array([],dtype=int)
            Z_logical2_Graph_Xerr=np.array([],dtype=int)

            id=0
            for i in range(0,Nx):
                for j in range(0,Ny):
                    if (i+j)%2==0:
                        Graph_Zerr.add_edge(v[i,j],v[(i+1)% Nx,(j+1)%Ny],qubit_id=id,weight=wZ,error_probability=pZ)
                        Graph_Xerr.add_edge(v[i,(j+1)%Ny],v[(i+1)%Nx,j],qubit_id=id,weight=wX,error_probability=pX)
                        id+=1
                        Graph_Zerr.add_edge(v[i,j],v[(i+1)%Nx,(j-1)%Ny],qubit_id=id,weight=wZ,error_probability=pZ)
                        Graph_Xerr.add_edge(v[i,(j-1)%Ny],v[(i+1)%Nx,j],qubit_id=id,weight=wX,error_probability=pX)
                        id+=1

            for i in range(0,Nx):
                for j in range(0,Ny):
                    if (i+j)%2==0 and i==0:
                        X_logical1_Graph_Zerr=np.append(X_logical1_Graph_Zerr,Graph_Zerr[v[i,j]][v[(i+1)%Nx,(j+1)%Ny]]['qubit_id'])                        
                        X_logical1_Graph_Zerr=np.append(X_logical1_Graph_Zerr,Graph_Zerr[v[i,j]][v[(i+1)%Nx,(j-1)%Ny]]['qubit_id'])                        
                    if (i+j)%2==0 and j==0:
                        X_logical2_Graph_Zerr=np.append(X_logical2_Graph_Zerr,Graph_Zerr[v[i,j]][v[(i+1)%Nx,(j+1)%Ny]]['qubit_id'])
                        X_logical2_Graph_Zerr=np.append(X_logical2_Graph_Zerr,Graph_Zerr[v[i,j]][v[(i-1)%Nx,(j+1)%Ny]]['qubit_id'])
                    if (i+j)%2==1 and i==0:
                        Z_logical2_Graph_Xerr=np.append(Z_logical2_Graph_Xerr,Graph_Xerr[v[i,j]][v[(i+1)%Nx,(j+1)%Ny]]['qubit_id'])
                        Z_logical2_Graph_Xerr=np.append(Z_logical2_Graph_Xerr,Graph_Xerr[v[i,j]][v[(i+1)%Nx,(j-1)%Ny]]['qubit_id'])
                    if (i+j)%2==1 and j==0:
                        Z_logical1_Graph_Xerr=np.append(Z_logical1_Graph_Xerr,Graph_Xerr[v[i,j]][v[(i+1)%Nx,(j+1)%Ny]]['qubit_id'])
                        Z_logical1_Graph_Xerr=np.append(Z_logical1_Graph_Xerr,Graph_Xerr[v[i,j]][v[(i-1)%Nx,(j+1)%Ny]]['qubit_id'])

            return [Graph_Xerr,Graph_Zerr,X_logical1_Graph_Zerr,X_logical2_Graph_Zerr,Z_logical1_Graph_Xerr,Z_logical2_Graph_Xerr]
    
        elif(Nx%2!=Ny%2):
            print('the modified decoder graphs and logicals (which are now a Y-Z or Y-X pair) needs to be implemented below')
            X_logical1_Graph_Zerr=np.array([],dtype=int)
            Z_logical1_Graph_Xerr=np.array([],dtype=int)

            id=0
            for i in range(0,Nx):
                for j in range(0,Ny):
                    if (i+j)%2==0:
                        Graph_Zerr.add_edge(v[i,j],v[(i+1)% Nx,(j+1)%Ny],qubit_id=id,weight=wZ,error_probability=pZ)
                        Graph_Xerr.add_edge(v[i,(j+1)%Ny],v[(i+1)%Nx,j],qubit_id=id,weight=wX,error_probability=pX)
                        id+=1
                        Graph_Zerr.add_edge(v[i,j],v[(i+1)%Nx,(j-1)%Ny],qubit_id=id,weight=wZ,error_probability=pZ)
                        Graph_Xerr.add_edge(v[i,(j-1)%Ny],v[(i+1)%Nx,j],qubit_id=id,weight=wX,error_probability=pX)
                        id+=1
            
            
            for i in range(0,Nx):
                for j in range(0,Ny):
                    if v[i,j] in range(0,Nx*Ny,Nx):
                        X_logical1_Graph_Zerr=np.append(X_logical1_Graph_Zerr,Graph_Zerr[v[i,j]][v[(i+1)%Nx,(j+1)%Ny],'qubit_id'])
                    if v[i,j] in range(0,Nx):
                        Z_logical1_Graph_Xerr=np.append(Z_logical1_Graph_Xerr,Graph_Xerr[v[i,j]][v[(i+1)%Nx,(j+1)%Ny],'qubit_id'])

            return [Graph_Xerr,Graph_Zerr,X_logical1_Graph_Zerr,Z_logical1_Graph_Xerr]



In [798]:
prob_dist=[0.1,0.3,0.3,0.3]
[Graph_Xerr,Graph_Zerr,X_logical1_Graph_Zerr,Z_logical1_Graph_Xerr]=construct_rot_lattice(4,4,prob_dist,0)


for u,v,w in Graph_Xerr.edges:
    print(u,v,w,Graph_Xerr[u][v][w]['qubit_id'])
print(X_logical1_Graph_Zerr)
for u,v,w in Graph_Zerr.edges:
    print(u,v,Graph_Zerr[u][v][w]['qubit_id'])

print(len(Graph_Xerr.edges),len(Graph_Zerr.edges))
print(Z_logical1_Graph_Xerr)


id= 25
3 0 0 0
3 6 0 3
0 12 0 1
0 12 1 5
0 4 0 7
12 1 0 11
12 1 1 15
12 2 0 21
12 12.0 0 -1
9 6 0 2
9 13 0 4
6 4 0 6
6 10 0 9
13 10 0 8
13 10 1 14
13 11 0 18
13 11 1 24
4 1 0 10
4 7 0 13
10 7 0 12
1 5 0 17
7 5 0 16
7 11 0 19
5 2 0 20
5 8 0 23
11 8 0 22
[5 6 7 8 9]
12 4 0
12 4 3
12 0 1
12 8 2
12 8 4
12 12.0 -1
4 6 6
4 2 7
0 2 5
8 10 8
8 6 9
2 5 10
2 1 11
6 9 12
6 5 13
10 9 14
5 7 16
5 3 17
1 3 15
9 11 18
9 7 19
3 13 20
3 13 21
7 13 22
7 13 23
11 13 24
26 26
[ 7  0 17 10 20]


In [799]:
Graph_Xerr.edges

MultiEdgeView([(3, 0, 0), (3, 6, 0), (0, 12, 0), (0, 12, 1), (0, 4, 0), (12, 1, 0), (12, 1, 1), (12, 2, 0), (12, 12.0, 0), (9, 6, 0), (9, 13, 0), (6, 4, 0), (6, 10, 0), (13, 10, 0), (13, 10, 1), (13, 11, 0), (13, 11, 1), (4, 1, 0), (4, 7, 0), (10, 7, 0), (1, 5, 0), (7, 5, 0), (7, 11, 0), (5, 2, 0), (5, 8, 0), (11, 8, 0)])

In [800]:
mXerr=Matching(Graph_Xerr)

In [791]:
mXerr

<pymatching.Matching object with 25 qubits, 13 stabilisers, 1 boundary node, and 26 edges>

In [792]:
mZerr=Matching(Graph_Zerr)

        
#noise
noiseXerr,syndromeXerr=mXerr.add_noise()
noiseZerr,syndromeZerr=mZerr.add_noise()

#correction
correctionXerr=mXerr.decode(syndromeXerr)
correctionZerr=mZerr.decode(syndromeZerr)

#correction+noise 
corr_noise_Xerr=np.array((correctionXerr+noiseXerr)%2)
corr_noise_Zerr=np.array((correctionZerr+noiseZerr)%2)


In [819]:
x=np.random.rand(10,6,3)
x

array([[[2.96001730e-01, 5.10137001e-01, 5.98621047e-01],
        [5.49753445e-01, 2.55207194e-01, 2.64928852e-01],
        [1.04906227e-01, 7.94340762e-01, 2.05845104e-01],
        [7.41907933e-01, 5.73448417e-01, 3.48998510e-02],
        [4.04990802e-02, 7.49846323e-01, 2.35904134e-01],
        [4.34088099e-01, 7.41232061e-01, 8.94067925e-01]],

       [[4.57259169e-01, 3.04953246e-01, 7.00707246e-01],
        [5.14085971e-02, 7.36976318e-01, 2.51368084e-01],
        [3.83831705e-01, 8.37119495e-01, 7.76879378e-01],
        [3.03515872e-01, 4.56676507e-02, 3.87215629e-01],
        [5.58064403e-01, 3.41396598e-02, 3.29183128e-01],
        [8.01596795e-01, 4.75162912e-01, 7.72511033e-01]],

       [[8.45912332e-01, 6.48863657e-01, 2.89955108e-01],
        [5.58763194e-01, 2.12993456e-01, 7.72986522e-01],
        [2.47548954e-01, 2.04605544e-01, 5.75633038e-01],
        [8.94262688e-01, 2.22624154e-01, 4.92221796e-01],
        [6.73073534e-01, 7.99895900e-02, 5.63055364e-01],
        [5

In [820]:
np.sum(x,axis=0)

array([[4.18327886, 5.89657704, 5.78667029],
       [4.50839921, 3.52741388, 5.13267015],
       [4.12412956, 6.17270762, 5.02394619],
       [5.64278161, 4.76025871, 4.33278808],
       [4.51146396, 3.69403209, 5.3843456 ],
       [5.24462262, 5.41102565, 4.85544421]])

In [815]:
0.71386226+0.28+0.07+0.14+0.152

1.35586226

In [822]:
[vX,vZ]=planardic_obc(Nx,Ny)
for i in range(1,Nx+1):
    for j in range(1,Ny+1):
        if j==Ny:
            hadamard_vec[Graph_Zerr[vZ[i,j]][vZ[i+1,j]][w]['qubit_id']]=1
        else:
            hadamard_vec[Graph_Zerr[vZ[i,j]][vZ[i+1,j]][w]['qubit_id']]=1
        if i==1:
            hadamard_vec[Graph_Zerr[vZ[i-1,j]][vZ[i,j]][w]['qubit_id']]=1
for i in range(0,Nx+1):
    for j in range(1,Ny+1):
        if j==1 and i%2==0:
            hadamard_vec[Graph_Zerr[vZ[i,j]][vZ[i+1,j]][w]['qubit_id']]=(hadamard_vec[Graph_Zerr[vZ[i,j]][vZ[i+1,j]][w]['qubit_id']]+1)%2
        if j==Ny and i%2==1:
            hadamard_vec[Graph_Zerr[vZ[i,j]][vZ[i+1,j]][w]['qubit_id']]=(hadamard_vec[Graph_Zerr[vZ[i,j]][vZ[i+1,j]][w]['qubit_id']]+1)%2


NameError: name 'planardic_obc' is not defined

In [823]:

def planardic_obc(Nx,Ny):

    Zerr_syndrome_num,Xerr_syndrome_num=Nx*Ny,(Nx+1)*(Ny-1)
    virZ1,virZ2=Zerr_syndrome_num,Zerr_syndrome_num+1
    virX1,virX2=Xerr_syndrome_num,Xerr_syndrome_num+1

    vX=np.zeros((Nx+2,Ny+2),dtype=int)

    for i in range(0,Nx+2):
        for j in range(0,Ny+2):
            if j==0:
                vX[i,j]=virX1
            elif j==Ny:
                vX[i,j]=virX2
            else:
                vX[i,j]=i-1+(j-1)*(Nx+1)

    vZ=np.zeros((Nx+2,Ny+2),dtype=int)

    for i in range(0,Nx+2):
        for j in range(1,Ny+1):
            if i==0:
                vZ[i,j]=virZ1
            elif i==Nx+1:
                vZ[i,j]=virZ2
            else:
                vZ[i,j]=i-1+(j-1)*Nx
    return [vX,vZ]


def graph_temp(Nx,Ny,prob_dist,periodic,rotated):
  
    if rotated==1:
        pI,pX,pY,pZ=prob_dist
        #weights
        wZ=abs(np.log(pZ/pI))
        wX=abs(np.log(pX/pI))

        if periodic==0:
            #even times even stabilizer number lattice
            Zerr_syndrome_num=(Nx/2+1)*Ny
            virZ1,virZ2=Zerr_syndrome_num,Zerr_syndrome_num+1
            [vX,vZ]=rotdic_obc(Nx,Ny)
            Graph_Zerr=nx.MultiGraph()
            id=0
            for i in range(1,Nx+3):
                for j in range(1,Ny+3): 
                    if (i+j)%2==1:
                        if vZ[i+1,j+1]>-1:
                            Graph_Zerr.add_edge(vZ[i,j],vZ[i+1,j+1],qubit_id=id,weight=wZ,error_probability=pZ)
                            id+=1
                        if vZ[i+1,j-1]>-1:
                            Graph_Zerr.add_edge(vZ[i,j],vZ[i+1,j-1],qubit_id=id,weight=wZ,error_probability=pZ)
                            id+=1
                        
            Graph_Zerr.add_edge(virZ1,virZ2,qubit_id=-1,weight=0,error_probability=0)  

            Graph_Zerr.nodes[virZ1]['is_boundary']=True
            Graph_Zerr.nodes[virZ2]['is_boundary']=True

        else: 
            v=rotdic_pbc(Nx,Ny)
            Graph_Zerr=nx.MultiGraph()  
            id=0                      
            if (Nx%2==0 and Ny%2==0):
                for i in range(0,Nx):
                    for j in range(0,Ny):
                        if (i+j)%2==0:
                            Graph_Zerr.add_edge(v[i,j],v[(i+1)% Nx,(j+1)%Ny],qubit_id=id,weight=wZ,error_probability=pZ)
                            id+=1
                            Graph_Zerr.add_edge(v[i,j],v[(i+1)%Nx,(j-1)%Ny],qubit_id=id,weight=wZ,error_probability=pZ)
                            id+=1

    else:
        pI,pX,pY,pZ=prob_dist
        #weights
        wZ=abs(np.log(pZ/pI))
        wX=abs(np.log(pX/pI))

        Graph_Zerr=nx.MultiGraph()

        id=0
        if(periodic==1):
            [vX,vZ]=planardic_pbc(Nx,Ny)
            for i in range(0,Nx):
                for j in range(0,Ny):
                    Graph_Zerr.add_edge(vZ[i][j],vZ[(i+1)%Nx][j],qubit_id=id,weight=wZ,error_probability=pZ)
                    id+=1

                    Graph_Zerr.add_edge(vZ[i][j],vZ[i][(j+1)%Ny],qubit_id=id,weight=wZ,error_probability=pZ)
                    id+=1
        else:
            virZ1,virZ2=Nx*Ny,Nx*Ny+1
            [vX,vZ]=planardic_obc(Nx,Ny)
            for i in range(1,Nx+1):
                for j in range(1,Ny+1):
                    if j==Ny:
                        Graph_Zerr.add_edge(vZ[i,j],vZ[i+1,j],qubit_id=id,weight=wZ,error_probability=pZ)  
                        id+=1
                    else:
                        Graph_Zerr.add_edge(vZ[i,j],vZ[i+1,j],qubit_id=id,weight=wZ,error_probability=pZ)  
                        id+=1
                        Graph_Zerr.add_edge(vZ[i,j],vZ[i,j+1],qubit_id=id,weight=wZ,error_probability=pZ) 
                        id+=1
                    if i==1:
                        Graph_Zerr.add_edge(vZ[i-1,j],vZ[i,j],qubit_id=id,weight=wZ,error_probability=pZ)  
                        id+=1

            Graph_Zerr.add_edge(virZ1,virZ2,qubit_id=-1,weight=0,error_probability=0)    

    return Graph_Zerr



In [828]:
Nx=4
Ny=4
# prob_dist
rotated=0
Graph_Zerr= graph_temp(Nx,Ny,prob_dist,periodic,rotated)

In [837]:
[vX,vZ]=planardic_obc(Nx,Ny)
hadamard_vec=np.zeros(len(Graph_Zerr.edges)-1)
for i in range(1,Nx+1):
    for j in range(1,Ny+1):
        if j==Ny:
            hadamard_vec[Graph_Zerr[vZ[i,j]][vZ[i+1,j]][w]['qubit_id']]=1
        else:
            hadamard_vec[Graph_Zerr[vZ[i,j]][vZ[i+1,j]][w]['qubit_id']]=1
        if i==1:
            hadamard_vec[Graph_Zerr[vZ[i-1,j]][vZ[i,j]][w]['qubit_id']]=1
for i in range(0,Nx+1):
    for j in range(1,Ny+1):
        if j==1 and i%2==1:
            hadamard_vec[Graph_Zerr[vZ[i,j]][vZ[i+1,j]][w]['qubit_id']]=(hadamard_vec[Graph_Zerr[vZ[i,j]][vZ[i+1,j]][w]['qubit_id']]+1)%2
        if j==Ny and i%2==0:
            hadamard_vec[Graph_Zerr[vZ[i,j]][vZ[i+1,j]][w]['qubit_id']]=(hadamard_vec[Graph_Zerr[vZ[i,j]][vZ[i+1,j]][w]['qubit_id']]+1)%2


In [838]:
np.nonzero(hadamard_vec)

(array([ 2,  3,  5,  6,  8,  9, 11, 13, 15, 20, 22, 24, 25, 27, 29]),)

In [834]:
for u,v,w in Graph_Zerr.edges:
    print(u,v,w,Graph_Zerr[u][v][w]['qubit_id'])

0 1 0 0
0 4 0 1
0 16 0 2
1 2 0 11
1 5 0 12
4 5 0 3
4 8 0 4
4 16 0 5
16 8 0 8
16 12 0 10
16 17 0 -1
5 6 0 13
5 9 0 14
8 9 0 6
8 12 0 7
9 10 0 15
9 13 0 16
12 13 0 9
13 14 0 17
2 3 0 18
2 6 0 19
6 7 0 20
6 10 0 21
10 11 0 22
10 14 0 23
14 15 0 24
3 17 0 25
3 7 0 26
7 17 0 27
7 11 0 28
11 17 0 29
11 15 0 30
15 17 0 31


In [5]:
import numpy as np
x=np.random.rand(5)

In [6]:
x

array([0.72095145, 0.95077429, 0.38168001, 0.66100647, 0.9490469 ])

In [8]:
x[4:5]

array([0.9490469])

In [16]:
n_qubits=10
import qecsim
from qecsim import app
from qecsim.models.generic import PhaseFlipErrorModel,DepolarizingErrorModel,BiasedDepolarizingErrorModel
from qecsim import paulitools as pt

prob_dist=[0.1,0.3,0.3,0.3]
rng = np.random.default_rng()
error_Pauli = rng.choice(('I', 'X', 'Y', 'Z'),size=n_qubits,p=prob_dist)
step_error=pt.pauli_to_bsf(''.join(error_Pauli))
noiseXerr= step_error[0:n_qubits]
noiseZerr=step_error[n_qubits:-1]
print(error_Pauli,noiseXerr,noiseZerr)

['Z' 'Y' 'X' 'I' 'Z' 'X' 'Y' 'Y' 'X' 'I'] [0 1 1 0 0 1 1 1 1 0] [1 1 0 0 1 0 1 1 0]


In [ ]:
def noisesyndrome(Graph_Xerr,Graph_Zerr,prob_dist,n_qubits):

    rng = np.random.default_rng()
    error_Pauli = rng.choice(('I', 'X', 'Y', 'Z'),size=n_qubits,p=prob_dist)
    step_error=pt.pauli_to_bsf(''.join(error_Pauli))
    noiseXerr= step_error[0:n_qubits]
    noiseZerr=step_error[n_qubits:-1]

    syndromeXerr=np.zeros(len(Graph_Xerr.nodes))
    syndromeZerr=np.zeros(len(Graph_Zerr.nodes))

    for u,v,w in Graph_Zerr.edges:
        if noiseZerr[Graph_Zerr[u][v][w]['qubit_id']]==1:
            syndromeZerr[u]=(syndromeZerr[u]+1)%2
            syndromeZerr[v]=(syndromeZerr[v]+1)%2
            
    for u,v,w in Graph_Xerr.edges:
        if noiseXerr[Graph_Xerr[u][v][w]['qubit_id']]==1:
            syndromeXerr[u]=(syndromeXerr[u]+1)%2
            syndromeXerr[v]=(syndromeXerr[v]+1)%2      

    return noiseXerr,syndromeXerr,noiseZerr,syndromeZerr   